# Coursera Capstone Project Notebook

In [2]:
import pandas as pd
import numpy as np

In [3]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Capstone Project: Battle of the Neighborhoods - Week 1
### 1. Introduction/Business Problem

A young professional, new to London, United Kingdom, is looking for a neighborhood to move to. They are looking at 3 particular areas in North London: Holland Park and Notting Hill, Angel and Islington, and Shoreditch and Aldgate (which could be grouped into their respective Boroughs).
They are looking for an area with good rent, and lots of amenities such as cafes, gyms, bars, etc, nearby. As a newcomer to London, they are hoping that they will be surrounded by residents aroudn their age, mid 20s. They are hoping also for good transportation links to their new job near London Bridge, as well as to central London.

This will be useful to compare different neighborhoods with one another to see which one best fits with the users' specfic interests. With additional analysis on local schools, it would also be beneficial for families looking for neighborhoods to settle into.

### 2. Data needed

In order to determine the best neighborhood for our user, we will be using Foursquare location data of London to determine the venues nearby these three areas. We can group the venues by type, such as cafes, gyms, grocery stores, and subcategories such as type of cuisine for resturants or brunch offerings, and determine how many of each there are. Further analysis can be done to determine popularity of venues during certain times, such as Thursday to Saturday evenings which are known to return a high turnout of young people at particular venues such as bars and clubs. A dataset containing London rent prices can be found and, and average rent of a double bedroom can also be determined to see which neighborhood best fits with the user's budget. Transportation data can also be used to determine the number of major bus or tube routes that pass through the neighborhood to get to London Bridge and/or Central London, in order to determine how well connected the neighorhood is.


### 3. Gathering Data

In [4]:
#import libraries needed
import pandas as pd
import numpy as np

import json
import requests
from pandas.io.json import json_normalize
print('Libraries imported.')

Libraries imported.


In [5]:
#importing other libraries
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [6]:
# Get Foursquare credentials
CLIENT_ID = 'CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3' # your Foursquare ID
CLIENT_SECRET = 'OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3
CLIENT_SECRET:OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0


In [7]:
#Get London data
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


#### import London Neighbourhood/Borough Data


In [8]:
#1. Web Scraping
# library for BeautifulSoup
from bs4 import BeautifulSoup
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
wikipedia_page = requests.get(wikipedia_link)

In [9]:
# Cleans html file
soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
# This extracts the "tbody" within the table where class is "wikitable sortable"
table = soup.find('table', {'class':'wikitable sortable'}).tbody
# Extracts all "tr" (table rows) within the table above
rows = table.find_all('tr')
# Extracts the column headers, removes and replaces possible '\n' with space for the "th" tag
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
# Converts columns to pd dataframe
df = pd.DataFrame(columns = columns)
'''
Extracts every row with corresponding columns then appends the values to the create pd dataframe "df". The first row (row[0]) is skipped because it is already the header
'''
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)
        df

In [10]:
#view dataframe
print(df.shape)
df.head()

(533, 6)


,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [11]:
# Remove Borough reference numbers with []
df.iloc[:,1]=df.iloc[:,1].str.rstrip('[ 0123456789]')
df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon,CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon,CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [12]:
df.iloc[:,1].head()

0                Bexley,  Greenwich
1    Ealing, Hammersmith and Fulham
2                           Croydon
3                           Croydon
4                            Bexley
Name: London borough, dtype: object

In [13]:
#constant error with calling certain columns, indentified issue with following code:
print(df.columns.tolist())

['Location', 'London\xa0borough', 'Post town', 'Postcode\xa0district', 'Dial\xa0code', 'OS grid ref']


In [14]:
#renaming problem columns
df=df.rename(columns={"London\xa0borough": "London Borough", "Postcode\xa0district": "Postcode District", "Dial\xa0code":"Dial Code"})
df['London Borough'].head()

0                Bexley,  Greenwich
1    Ealing, Hammersmith and Fulham
2                           Croydon
3                           Croydon
4                            Bexley
Name: London Borough, dtype: object

In [15]:
#split multiple elements into individual rows
LB_split = pd.DataFrame(df['London Borough'].str.split(',').tolist(),index=df['Location']).stack()
PT_split = pd.DataFrame(df['Post town'].str.split(',').tolist(), index=df['Location']).stack()
PD_split = pd.DataFrame(df['Postcode District'].str.split(',').tolist(), index=df['Location']).stack()

In [16]:
#London Borough Split
LB_split

Location                                     
Abbey Wood                                  0                     Bexley
                                            1                  Greenwich
Acton                                       0                     Ealing
                                            1     Hammersmith and Fulham
Addington                                   0                    Croydon
Addiscombe                                  0                    Croydon
Albany Park                                 0                     Bexley
Aldborough Hatch                            0                  Redbridge
Aldgate                                     0                       City
Aldwych                                     0                Westminster
Alperton                                    0                      Brent
Anerley                                     0                    Bromley
Angel                                       0                  Islington
Aperf

In [17]:
#London Borough reindex and clean
LB_split = LB_split.reset_index([0, 'Location'])
LB_split.columns = ['Location', 'London Borough']
LB_split.head()

,Location,London Borough
0,Abbey Wood,Bexley
1,Abbey Wood,Greenwich
2,Acton,Ealing
3,Acton,Hammersmith and Fulham
4,Addington,Croydon


In [18]:
#post town split
PT_split

Location                                     
Abbey Wood                                  0             LONDON
Acton                                       0             LONDON
Addington                                   0            CROYDON
Addiscombe                                  0            CROYDON
Albany Park                                 0             BEXLEY
                                            1             SIDCUP
Aldborough Hatch                            0             ILFORD
Aldgate                                     0             LONDON
Aldwych                                     0             LONDON
Alperton                                    0            WEMBLEY
Anerley                                     0             LONDON
Angel                                       0             LONDON
Aperfield                                   0          WESTERHAM
Archway                                     0             LONDON
Ardleigh Green                              

In [19]:
#post town reindex and clean
PT_split = PT_split.reset_index([0, 'Location'])
PT_split.columns = ['Location', 'Post town']
PT_split.head()

,Location,Post town
0,Abbey Wood,LONDON
1,Acton,LONDON
2,Addington,CROYDON
3,Addiscombe,CROYDON
4,Albany Park,BEXLEY


In [ ]:
#postcode district split
PD_split

In [21]:
#postcode district reindec and clean
PD_split = PD_split.reset_index([0, 'Location'])
PD_split.columns = ['Location', 'Postcode District']
PD_split.head()

,Location,Postcode District
0,Abbey Wood,SE2
1,Acton,W3
2,Acton,W4
3,Addington,CR0
4,Addiscombe,CR0


In [ ]:
#merge new split dataframes together
LD_merged=pd.merge(LB_split,PT_split, how='outer', on='Location')
LD_merged.head(11)


In [23]:
LDN_merged=pd.merge(LD_merged,PD_split, how='outer', on='Location')
LDN_merged #new dataframe

,Location,London Borough,Post town,Postcode District
0,Abbey Wood,Bexley,LONDON,SE2
1,Abbey Wood,Greenwich,LONDON,SE2
2,Acton,Ealing,LONDON,W3
3,Acton,Ealing,LONDON,W4
4,Acton,Hammersmith and Fulham,LONDON,W3
5,Acton,Hammersmith and Fulham,LONDON,W4
6,Addington,Croydon,CROYDON,CR0
7,Addiscombe,Croydon,CROYDON,CR0
8,Albany Park,Bexley,BEXLEY,DA5
9,Albany Park,Bexley,BEXLEY,DA14


In [24]:
LDN_merged['London Borough']


0                       Bexley
1                    Greenwich
2                       Ealing
3                       Ealing
4       Hammersmith and Fulham
5       Hammersmith and Fulham
6                      Croydon
7                      Croydon
8                       Bexley
9                       Bexley
10                      Bexley
11                      Bexley
12                   Redbridge
13                        City
14                 Westminster
15                       Brent
16                     Bromley
17                   Islington
18                   Islington
19                     Bromley
20                   Islington
21                    Havering
22                      Barnet
23                      Barnet
24                      Barnet
25                      Barnet
26                     Enfield
27                     Enfield
28                  Wandsworth
29                   Southwark
                ...           
758     Hammersmith and Fulham
759     

In [25]:
LDN_merged

,Location,London Borough,Post town,Postcode District
0,Abbey Wood,Bexley,LONDON,SE2
1,Abbey Wood,Greenwich,LONDON,SE2
2,Acton,Ealing,LONDON,W3
3,Acton,Ealing,LONDON,W4
4,Acton,Hammersmith and Fulham,LONDON,W3
5,Acton,Hammersmith and Fulham,LONDON,W4
6,Addington,Croydon,CROYDON,CR0
7,Addiscombe,Croydon,CROYDON,CR0
8,Albany Park,Bexley,BEXLEY,DA5
9,Albany Park,Bexley,BEXLEY,DA14


In [26]:
#importing second dataset: with Borough Coordinates
from bs4 import BeautifulSoup
wikipedia_link2 = 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
wikipedia_page2 = requests.get(wikipedia_link2)

# Cleans html file
soup = BeautifulSoup(wikipedia_page2.content, 'html.parser')
# This extracts the "tbody" within the table where class is "wikitable sortable"
table = soup.find('table', {'class':'wikitable sortable'}).tbody
# Extracts all "tr" (table rows) within the table above
rows = table.find_all('tr')
# Extracts the column headers, removes and replaces possible '\n' with space for the "th" tag
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
# Converts columns to pd dataframe
df2 = pd.DataFrame(columns = columns)
'''
Extracts every row with corresponding columns then appends the values to the create pd dataframe "df". The first row (row[0]) is skipped because it is already the header
'''
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df2 = df2.append(pd.Series(values, index = columns), ignore_index = True)
        df2
        
df2.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
1,Barnet,,,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20


In [27]:
# Remove Borough reference numbers with []
df2['Borough']=df2['Borough'].str.rstrip('[ 0123456789]')
df2.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
1,Barnet,,,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20


In [28]:
#checking column names
print(df2.columns.tolist())

['Borough', 'Inner', 'Status', 'Local authority', 'Political control', 'Headquarters', 'Area (sq mi)', 'Population (2013 est)[1]', 'Co-ordinates', ' Nr. in map ']


In [29]:
#extracting needed info
#drop unecessary columns
df2_a=df2.drop(['Inner','Status','Local authority','Political control','Headquarters','Area (sq mi)', 'Population (2013 est)[1]', ' Nr. in map '], axis=1)
df2_a

,Borough,Co-ordinates
0,Barking and Dagenham [note,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...
1,Barnet,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...
2,Bexley,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...
3,Brent,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...
4,Bromley,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...
5,Camden,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...
6,Croydon,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...
7,Ealing,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...
8,Enfield,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...
9,Greenwich [note,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...


In [30]:
#edit Borough values that contain unnecessary strings
df2_a.replace(to_replace='Barking and Dagenham [note', value='Barking and Dagenham', inplace=True)
df2_a.replace(to_replace='Greenwich [note', value='Greenwich', inplace=True)
df2_a.replace(to_replace='Hammersmith and Fulham [note', value='Hammersmith and Fulham [note', inplace=True)
df2_a

,Borough,Co-ordinates
0,Barking and Dagenham,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...
1,Barnet,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...
2,Bexley,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...
3,Brent,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...
4,Bromley,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...
5,Camden,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...
6,Croydon,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...
7,Ealing,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...
8,Enfield,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...
9,Greenwich,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...


In [31]:
#rename column Borough
df2_a.rename(columns={"Borough": "London Borough"}, inplace=True)
df2_a

,London Borough,Co-ordinates
0,Barking and Dagenham,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...
1,Barnet,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...
2,Bexley,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...
3,Brent,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...
4,Bromley,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...
5,Camden,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...
6,Croydon,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...
7,Ealing,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...
8,Enfield,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...
9,Greenwich,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...


In [32]:
LDN_merged['London Borough'].unique()

array(['Bexley', '  Greenwich', 'Ealing', ' Hammersmith and Fulham',
       'Croydon', 'Redbridge', 'City', 'Westminster', 'Brent', 'Bromley',
       'Islington', 'Havering', 'Barnet', 'Enfield', 'Wandsworth',
       'Southwark', 'Barking and Dagenham', 'Richmond upon Thames',
       'Newham', 'Sutton', 'Lewisham', 'Harrow', 'Camden',
       'Kingston upon Thames', 'Tower Hamlets', 'Greenwich', 'Haringey',
       'Hounslow', 'Lambeth',
       'Kensington and ChelseaHammersmith and Fulham', 'Waltham Forest',
       ' Barking and Dagenham', 'Kensington and Chelsea', ' Ealing',
       ' Wandsworth', ' Enfield', 'Merton', 'Hillingdon', ' Brent',
       ' Camden', 'Hackney', 'Dartford', ' Greenwich', 'Islington & City',
       ' Islington', 'Hammersmith and Fulham', ' Lewisham', ' Southwark',
       ' Harrow', 'Camden and Islington', 'Haringey and Barnet',
       ' Bromley', ' Waltham Forest', ' Westminster',
       ' Kingston upon Thames'], dtype=object)

In [33]:
#realizing various element names, removing leading spaces
LDN_merged['London Borough']=LDN_merged['London Borough'].str.lstrip(' ')
LDN_merged['London Borough'].unique()

array(['Bexley', 'Greenwich', 'Ealing', 'Hammersmith and Fulham',
       'Croydon', 'Redbridge', 'City', 'Westminster', 'Brent', 'Bromley',
       'Islington', 'Havering', 'Barnet', 'Enfield', 'Wandsworth',
       'Southwark', 'Barking and Dagenham', 'Richmond upon Thames',
       'Newham', 'Sutton', 'Lewisham', 'Harrow', 'Camden',
       'Kingston upon Thames', 'Tower Hamlets', 'Haringey', 'Hounslow',
       'Lambeth', 'Kensington and ChelseaHammersmith and Fulham',
       'Waltham Forest', 'Kensington and Chelsea', 'Merton', 'Hillingdon',
       'Hackney', 'Dartford', 'Islington & City', 'Camden and Islington',
       'Haringey and Barnet'], dtype=object)

In [34]:
LDN_merged['Location'].unique()

array(['Abbey Wood', 'Acton', 'Addington', 'Addiscombe', 'Albany Park',
       'Aldborough Hatch', 'Aldgate', 'Aldwych', 'Alperton', 'Anerley',
       'Angel', 'Aperfield', 'Archway', 'Ardleigh Green', 'Arkley',
       'Arnos Grove', 'Balham', 'Bankside', 'Barbican', 'Barking',
       'Barkingside', 'Barnehurst', 'Barnes', 'Barnes Cray',
       'Barnet Gate', 'Barnet (also Chipping Barnet, High Barnet)',
       'Barnsbury', 'Battersea', 'Bayswater', 'Beckenham', 'Beckton',
       'Becontree', 'Becontree Heath', 'Beddington', 'Bedford Park',
       'Belgravia', 'Bellingham', 'Belmont', 'Belsize Park', 'Belvedere',
       'Bermondsey', 'Berrylands', 'Bethnal Green',
       'Bexley (also Old Bexley, Bexley Village)',
       'Bexleyheath (also Bexley New Town)', 'Bickley', 'Biggin Hill',
       'Blackfen', 'Blackfriars', 'Blackheath',
       'Blackheath Royal Standard', 'Blackwall', 'Blendon', 'Bloomsbury',
       'Botany Bay', 'Bounds Green', 'Bow', 'Bowes Park', 'Brentford',
       'Bren

In [35]:
#identifying rows with neighborhoods of interest: Notting Hill, Islington, Angel, Shoreditch, Spitafields to find wider Borough later
print(LDN_merged.loc[LDN_merged['Location']=='Angel'])
print(LDN_merged.loc[LDN_merged['Location']=='Aldgate'])
print(LDN_merged.loc[LDN_merged['Location']=='Islington'])
print(LDN_merged.loc[LDN_merged['Location']=='Notting Hill'])
print(LDN_merged.loc[LDN_merged['Location']=='Shoreditch'])
print(LDN_merged.loc[LDN_merged['Location']=='Spitalfields'])

   Location London Borough Post town Postcode District
17    Angel      Islington    LONDON               EC1
18    Angel      Islington    LONDON                N1
   Location London Borough Post town Postcode District
13  Aldgate           City    LONDON               EC3
      Location London Borough Post town Postcode District
388  Islington      Islington    LONDON                N1
         Location          London Borough Post town Postcode District
496  Notting Hill  Kensington and Chelsea    LONDON               W11
       Location London Borough Post town Postcode District
594  Shoreditch        Hackney    LONDON                N1
         Location London Borough Post town Postcode District
624  Spitalfields  Tower Hamlets    LONDON                E1


In [36]:
#the Boroughs we shall be investigating are Islington, Kensington and Chelsea, Hackney, Tower Hamlets - indentifeid the indexers as well
#City is an error for aldgate; aldgate is in Tower Hamlets as well

In [37]:
#getting coordinates for Boroughs of interest to build dataframe using GEOPY
address = 'Islington, London'

geolocator = Nominatim(user_agent="Islington_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Islington are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Islington are 51.5384287, -0.0999051.


In [38]:
address = 'Tower Hamlets, London'

geolocator = Nominatim(user_agent="Tower_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tower Hamlets are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tower Hamlets are 51.5256294, -0.0335853.


In [39]:
address = 'Kensington and Chelsea, London'

geolocator = Nominatim(user_agent="Kensington_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kensington and Chelsea are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kensington and Chelsea are 51.4989948, -0.1991229.


In [40]:
address = 'Hackney, London'

geolocator = Nominatim(user_agent="Hackney_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hackney are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hackney are 51.5432402, -0.0493621.


In [41]:
#compare to df2_a
print(df2_a.loc[df2_a['London Borough']=='Islington'])
print(df2_a.loc[df2_a['London Borough']=='Tower Hamlets'])
print(df2_a.loc[df2_a['London Borough']=='Kensington and Chelsea'])
print(df2_a.loc[df2_a['London Borough']=='Hackney'])

   London Borough                                       Co-ordinates
17      Islington  51°32′30″N 0°06′08″W﻿ / ﻿51.5416°N 0.1022°W﻿ /...
   London Borough                                       Co-ordinates
28  Tower Hamlets  51°30′36″N 0°00′21″W﻿ / ﻿51.5099°N 0.0059°W﻿ /...
            London Borough                                       Co-ordinates
18  Kensington and Chelsea  51°30′07″N 0°11′41″W﻿ / ﻿51.5020°N 0.1947°W﻿ /...
   London Borough                                       Co-ordinates
10        Hackney  51°32′42″N 0°03′19″W﻿ / ﻿51.5450°N 0.0553°W﻿ /...


In [42]:
# there is discrepancies between the two data sets
#but building abbreviated dataframe for Foursquare with both datasets to see differences
Coord={'Borough':['Hackney','Islington','Kensington and Chelsea','Tower Hamlets'],\
       'Latitude-Geopy':[51.5432402,51.5384287,51.4989948,51.5256294],\
      'Longitude-Geopy':[-0.0493621,-0.0999051,-0.1991229,-0.0335853],\
      'Latitude-Wiki':[51.5450,51.5416,51.5020,51.5099],\
      'Longitude-Wiki':[-0.0553,-0.1022,-0.1947,-0.0059]}
Coord

{'Borough': ['Hackney',
  'Islington',
  'Kensington and Chelsea',
  'Tower Hamlets'],
 'Latitude-Geopy': [51.5432402, 51.5384287, 51.4989948, 51.5256294],
 'Longitude-Geopy': [-0.0493621, -0.0999051, -0.1991229, -0.0335853],
 'Latitude-Wiki': [51.545, 51.5416, 51.502, 51.5099],
 'Longitude-Wiki': [-0.0553, -0.1022, -0.1947, -0.0059]}

In [43]:
LDN_coord=pd.DataFrame(data=Coord)
LDN_coord

,Borough,Latitude-Geopy,Longitude-Geopy,Latitude-Wiki,Longitude-Wiki
0,Hackney,51.543240,-0.049362,51.5450,-0.0553
1,Islington,51.538429,-0.099905,51.5416,-0.1022
2,Kensington and Chelsea,51.498995,-0.199123,51.5020,-0.1947
3,Tower Hamlets,51.525629,-0.033585,51.5099,-0.0059


In [44]:
#create map of London Boroughs of interest using latitude and longitude values
map_LDN = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map to identify London boroughs/neighborhoods: GEOPY
for lat, lng, borough in zip(LDN_coord['Latitude-Geopy'],LDN_coord['Longitude-Geopy'],LDN_coord['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LDN)
    
map_LDN

In [45]:
# hackney and Tower Hamlets is too far out...

In [46]:
#create map of London Boroughs of interest using latitude and longitude values
map_LDN = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map to identify London boroughs/neighborhoods: WIKI
for lat, lng, borough in zip(LDN_coord['Latitude-Wiki'],LDN_coord['Longitude-Wiki'],LDN_coord['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LDN)
    
map_LDN

In [47]:
# "Tower Hamlets even worse in area" will try specific Neighbours - RESTART

In [48]:
#identifying rows with neighborhoods of interest: Notting Hill, Islington, Angel, Shoreditch, Spitafields to find wider Borough later
print(LDN_merged.loc[LDN_merged['Location']=='Angel'])
print(LDN_merged.loc[LDN_merged['Location']=='Aldgate'])
print(LDN_merged.loc[LDN_merged['Location']=='Islington'])
print(LDN_merged.loc[LDN_merged['Location']=='Notting Hill'])
print(LDN_merged.loc[LDN_merged['Location']=='Shoreditch'])
print(LDN_merged.loc[LDN_merged['Location']=='Spitalfields'])

   Location London Borough Post town Postcode District
17    Angel      Islington    LONDON               EC1
18    Angel      Islington    LONDON                N1
   Location London Borough Post town Postcode District
13  Aldgate           City    LONDON               EC3
      Location London Borough Post town Postcode District
388  Islington      Islington    LONDON                N1
         Location          London Borough Post town Postcode District
496  Notting Hill  Kensington and Chelsea    LONDON               W11
       Location London Borough Post town Postcode District
594  Shoreditch        Hackney    LONDON                N1
         Location London Borough Post town Postcode District
624  Spitalfields  Tower Hamlets    LONDON                E1


In [49]:
Coord2={'Neighborhood':['Angel','Islington','Notting Hill','Shoreditch','Spitalfields']}
LDN_hoods=pd.DataFrame(data=Coord2)
LDN_hoods

,Neighborhood
0,Angel
1,Islington
2,Notting Hill
3,Shoreditch
4,Spitalfields


In [50]:
#using GEOPY to find coordinates
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

addresses=['Angel, London', 'Islington, London', 'Notting Hill, London', 'Shoreditch, London', 'Spitalfields, London']

for address in addresses:
  g = geolocator.geocode(address)

  if g is None:
    print ('{} could not be geocoded'.format(address))
  else:
    print(g.address)
    print((g.latitude, g.longitude))

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0          conda-forge
    geopy:         2.0.0-pyh9f0ad1d_0 conda-forge


geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Angel, City Road, Angel, Finsbury, Lo

In [51]:
#creating df with geopy coordinates
hoods={'Latitude':[51.5319458,51.5384287,51.5109995,51.5266694,51.5195274], 'Longitude':[-0.1061056,-0.0999051,-0.2055267,-0.0798926,-0.0751701]}
LDN_hoodsc=pd.DataFrame(data=hoods)
LDN_hoodsc

,Latitude,Longitude
0,51.531946,-0.106106
1,51.538429,-0.099905
2,51.510999,-0.205527
3,51.526669,-0.079893
4,51.519527,-0.075170


In [52]:
LDN_ideal=LDN_hoods.join(LDN_hoodsc)
LDN_ideal

,Neighborhood,Latitude,Longitude
0,Angel,51.531946,-0.106106
1,Islington,51.538429,-0.099905
2,Notting Hill,51.510999,-0.205527
3,Shoreditch,51.526669,-0.079893
4,Spitalfields,51.519527,-0.075170


In [53]:
#create map of ideal London neighborhoods of interest using latitude and longitude values
map_LDN = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map to identify London boroughs/neighborhoods: WIKI
for lat, lng, neighborhood in zip(LDN_ideal['Latitude'],LDN_ideal['Longitude'],LDN_ideal['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LDN)
    
map_LDN

In [54]:
#much better in terms of where the user really wants to live

In [55]:
#using Foursquare to explore venues in each location
# Get Foursquare credentials
CLIENT_ID = 'CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3' # your Foursquare ID
CLIENT_SECRET = 'OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3
CLIENT_SECRET:OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0


In [56]:
#explore first neighborhood: Angel
neighborhood_latitude = LDN_ideal.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = LDN_ideal.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = LDN_ideal.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Angel are 51.5319458, -0.1061056.


In [57]:
#get top 100 venues
LIMIT = 100 

radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&v=20180604&ll=51.5319458,-0.1061056&radius=500&limit=100'

In [58]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f038e698005581f72d42fef'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Islington',
  'headerFullLocation': 'Islington, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 65,
  'suggestedBounds': {'ne': {'lat': 51.536445804500005,
    'lng': -0.09888528179333114},
   'sw': {'lat': 51.5274457955, 'lng': -0.11332591820666885}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50da842cc84c4c57279457da',
       'name': 'Present & Correct',
       'location': {'address': '23 Arlington Way',
        'lat': 51.53014701529403,
        'lng': -0.1062308090426986,
        'labeledLatLngs': [{'label': 'display',
          'lat': 5

In [59]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [60]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Present & Correct,Paper / Office Supplies Store,51.530147,-0.106231
1,The Three Johns,Pub,51.532825,-0.109186
2,Saint Espressō,Coffee Shop,51.531919,-0.107565
3,Virgin Active,Gym / Fitness Center,51.531343,-0.105135
4,Oseyo,Korean Restaurant,51.533823,-0.106810


In [61]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

65 venues were returned by Foursquare.


In [62]:
#explore first neighborhood: Islington
neighborhood_latitude = LDN_ideal.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = LDN_ideal.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = LDN_ideal.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Islington are 51.5384287, -0.0999051.


In [63]:
#get top 100 venues
LIMIT = 100 

radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&v=20180604&ll=51.5384287,-0.0999051&radius=500&limit=100'

In [64]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f03909e8a993d297202c186'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Islington',
  'headerFullLocation': 'Islington, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 69,
  'suggestedBounds': {'ne': {'lat': 51.5429287045,
    'lng': -0.09268375335963287},
   'sw': {'lat': 51.533928695499995, 'lng': -0.10712644664036712}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aec3461f964a52082c521e3',
       'name': 'Steve Hatt',
       'location': {'address': '88-90 Essex Rd',
        'lat': 51.53858794839484,
        'lng': -0.09904121479314358,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.538587

In [65]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Steve Hatt,Fish Market,51.538588,-0.099041
1,The Life Centre Islington,Yoga Studio,51.538056,-0.099171
2,Ottolenghi,Mediterranean Restaurant,51.539716,-0.102314
3,Little Angel Theatre,Theater,51.538759,-0.101295
4,Paper Mache Tiger,Boutique,51.539118,-0.100375


In [67]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

69 venues were returned by Foursquare.


In [68]:
#explore first neighborhood: Notting Hill
neighborhood_latitude = LDN_ideal.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = LDN_ideal.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = LDN_ideal.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Notting Hill are 51.5109995, -0.2055267.


In [69]:
#get top 100 venues
LIMIT = 100 

radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&v=20180604&ll=51.5109995,-0.2055267&radius=500&limit=100'

In [70]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f03909f8a993d297202c383'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kensington and Chelsea',
  'headerFullLocation': 'Kensington and Chelsea, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 55,
  'suggestedBounds': {'ne': {'lat': 51.5154995045, 'lng': -0.1983097020379208},
   'sw': {'lat': 51.506499495499995, 'lng': -0.2127436979620792}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5980bc24e65d0c3d5d28e3b2',
       'name': 'Core By Clare Smyth',
       'location': {'address': '92 Kensington Park Rd',
        'lat': 51.512651265916645,
        'lng': -0.20311585807061372,
        'labeledLatLngs': [{'label': 

In [71]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [72]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Core By Clare Smyth,Restaurant,51.512651,-0.203116
1,The Ladbroke Arms,Gastropub,51.509040,-0.202837
2,Stumper & Fielding,Boutique,51.513284,-0.202789
3,The Gold Bar,Bar,51.513018,-0.202531
4,Alexeeva & Jones Chocolate Boutique,Candy Store,51.513438,-0.203612


In [73]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

55 venues were returned by Foursquare.


In [74]:
#explore first neighborhood: Shoreditch
neighborhood_latitude = LDN_ideal.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = LDN_ideal.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = LDN_ideal.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Shoreditch are 51.5266694, -0.0798926.


In [75]:
#get top 100 venues
LIMIT = 200 

radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&v=20180604&ll=51.5266694,-0.0798926&radius=500&limit=200'

In [76]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f03900a8dec2f5f33f9da1c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Shoreditch',
  'headerFullLocation': 'Shoreditch, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 151,
  'suggestedBounds': {'ne': {'lat': 51.531169404500005,
    'lng': -0.07267311854597064},
   'sw': {'lat': 51.5221693955, 'lng': -0.08711208145402935}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d000afb21ea6ea817ce3f9f',
       'name': 'Happiness Forgets',
       'location': {'address': '8-9 Hoxton Sq',
        'crossStreet': 'Basement',
        'lat': 51.527746,
        'lng': -0.081591,
        'labeledLatLngs': [{'label': 'display',
 

In [77]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [78]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Happiness Forgets,Cocktail Bar,51.527746,-0.081591
1,National Centre for Circus Arts,Circus School,51.527578,-0.082551
2,Origin Coffee,Coffee Shop,51.526347,-0.081204
3,Homeslice Pizza,Pizza Place,51.527100,-0.079689
4,Artwords Bookshop,Bookstore,51.526236,-0.079702


In [79]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [80]:
#explore first neighborhood: Spitalfields
neighborhood_latitude = LDN_ideal.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = LDN_ideal.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = LDN_ideal.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Spitalfields are 51.5195274, -0.0751701.


In [81]:
#get top 100 venues
LIMIT = 100 

radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&v=20180604&ll=51.5195274,-0.0751701&radius=500&limit=100'

In [82]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f03907970e48376ff23cbb8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Spitalfields and Banglatown',
  'headerFullLocation': 'Spitalfields and Banglatown, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 39,
  'suggestedBounds': {'ne': {'lat': 51.5240274045, 'lng': -0.0679517507465596},
   'sw': {'lat': 51.5150273955, 'lng': -0.08238844925344041}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d2b2168342d6dcb732213cb',
       'name': 'Kastner & Ovens',
       'location': {'address': '13 Frying Pan Alley',
        'crossStreet': 'at Bell Lane',
        'lat': 51.517913,
        'lng': -0.076465,
        'labeledLa

In [83]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [84]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Kastner & Ovens,Café,51.517913,-0.076465
1,Old Spitalfields Market,Flea Market,51.519668,-0.075375
2,Gunpowder,Indian Restaurant,51.518436,-0.074732
3,Ottolenghi,Mediterranean Restaurant,51.518272,-0.077177
4,Taylor Taylor,Salon / Barbershop,51.520826,-0.074936


In [85]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

39 venues were returned by Foursquare.


In [86]:
#create code to find nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
#code to run function on ideal LDN neighborhoods
LDN_venues = getNearbyVenues(names=LDN_ideal['Neighborhood'],
                                   latitudes=LDN_ideal['Latitude'],
                                   longitudes=LDN_ideal['Longitude']
                                  )

Angel
Islington
Notting Hill
Shoreditch
Spitalfields


In [88]:
print(LDN_venues.shape)
LDN_venues.head()

(328, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Angel,51.531946,-0.106106,Present & Correct,51.530147,-0.106231,Paper / Office Supplies Store
1,Angel,51.531946,-0.106106,The Three Johns,51.532825,-0.109186,Pub
2,Angel,51.531946,-0.106106,Saint Espressō,51.531919,-0.107565,Coffee Shop
3,Angel,51.531946,-0.106106,Virgin Active,51.531343,-0.105135,Gym / Fitness Center
4,Angel,51.531946,-0.106106,Oseyo,51.533823,-0.106810,Korean Restaurant


In [89]:
LDN_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Angel,65,65,65,65,65,65
Islington,69,69,69,69,69,69
Notting Hill,55,55,55,55,55,55
Shoreditch,100,100,100,100,100,100
Spitalfields,39,39,39,39,39,39


In [90]:
print('There are {} uniques categories.'.format(len(LDN_venues['Venue Category'].unique())))

There are 113 uniques categories.


In [91]:
#view how many of each type of venue in each neighborhood
LDN_venues_grouped=LDN_venues.groupby(['Neighborhood','Venue Category']).count()
LDN_venues_grouped

Neighborhood Latitude  \
Neighborhood Venue Category                                         
Angel        Afghan Restaurant                                  1   
             Art Gallery                                        1   
             Arts & Crafts Store                                2   
             Austrian Restaurant                                1   
             Beer Bar                                           1   
             Boutique                                           1   
             Burger Joint                                       1   
             Burrito Place                                      2   
             Café                                               3   
             Canal                                              1   
             Canal Lock                                         1   
             Caucasian Restaurant                               1   
             Cheese Shop                                        1   
             Chocolate Shop                                     1   
             Coffee Shop                                        3   
             Comedy Club                                        1   
             Cupcake Shop                                       1   
             Farmers Market                                     1   
             Fish Market                                        1   
             Food Truck                                         2   
             French Restaurant                                  1   
             Gym / Fitness Center                               2   
             Hotel                                              2   
             Hunan Restaurant                                   1   
             Indian Restaurant                                  2   
             Japanese Restaurant                                1   
             Korean Restaurant                                  1   
             Mediterranean Restaurant                           2   
             Mexican Restaurant                                 2   
             Paper / Office Supplies Store                      1   
...                                                           ...   
Spitalfields Burger Joint                                       2   
             Café                                               2   
             Chocolate Shop                                     1   
             Clothing Store                                     1   
             Cocktail Bar                                       2   
             Coffee Shop                                        1   
             Event Space                                        1   
             Fish & Chips Shop                                  1   
             Flea Market                                        2   
             Food Stand                                         1   
             Food Truck                                         1   
             French Restaurant                                  1   
             Fried Chicken Joint                                1   
             Hotel                                              1   
             Indian Restaurant                                  2   
             Indie Movie Theater                                1   
             Market                                             1   
             Mediterranean Restaurant                           1   
             Mexican Restaurant                                 1   
             Middle Eastern Restaurant                          1   
             Organic Grocery                                    1   
             Park                                               1   
             Pilates Studio                                     1   
             Pub                                                1   
             Record Shop                                        1   
             Restaurant                   

In [92]:
#rest index
LDN_venues_grouped = LDN_venues_grouped.reset_index()
LDN_venues_grouped


,Neighborhood,Venue Category,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
0,Angel,Afghan Restaurant,1,1,1,1,1
1,Angel,Art Gallery,1,1,1,1,1
2,Angel,Arts & Crafts Store,2,2,2,2,2
3,Angel,Austrian Restaurant,1,1,1,1,1
4,Angel,Beer Bar,1,1,1,1,1
5,Angel,Boutique,1,1,1,1,1
6,Angel,Burger Joint,1,1,1,1,1
7,Angel,Burrito Place,2,2,2,2,2
8,Angel,Café,3,3,3,3,3
9,Angel,Canal,1,1,1,1,1


In [93]:
#checking column names
print(LDN_venues_grouped.columns.tolist())

['Neighborhood', 'Venue Category', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude']


In [94]:
#Analyze Each Neighborhood
# one hot encoding
LDN_onehot = pd.get_dummies(LDN_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
LDN_onehot['Neighborhood'] = LDN_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [LDN_onehot.columns[-1]] + list(LDN_onehot.columns[:-1])
LDN_onehot = LDN_onehot[fixed_columns]

print(LDN_onehot.shape)
LDN_onehot.head()


(328, 114)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,...,Sushi Restaurant,Tea Room,Theater,Thrift / Vintage Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Angel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Angel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Angel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Angel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
#frequency of occurrence of each category
LDN_grouped = LDN_onehot.groupby('Neighborhood').mean().reset_index()
LDN_grouped


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,...,Sushi Restaurant,Tea Room,Theater,Thrift / Vintage Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Angel,0.00,0.00,0.015385,0.000000,0.000000,0.015385,0.030769,0.00,0.000000,...,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.015385,0.000000,0.015385,0.015385
1,Islington,0.00,0.00,0.014493,0.000000,0.000000,0.000000,0.028986,0.00,0.000000,...,0.014493,0.000000,0.028986,0.014493,0.000000,0.000000,0.014493,0.000000,0.000000,0.014493
2,Notting Hill,0.00,0.00,0.000000,0.000000,0.018182,0.000000,0.000000,0.00,0.018182,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000
3,Shoreditch,0.01,0.01,0.000000,0.000000,0.000000,0.020000,0.010000,0.01,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.020000,0.010000,0.010000,0.010000
4,Spitalfields,0.00,0.00,0.000000,0.025641,0.000000,0.025641,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000


In [96]:
#checking column names
print(LDN_grouped.columns.tolist())

['Neighborhood', 'Accessories Store', 'Adult Boutique', 'Afghan Restaurant', 'African Restaurant', 'Antique Shop', 'Art Gallery', 'Arts & Crafts Store', 'Asian Restaurant', 'Australian Restaurant', 'Austrian Restaurant', 'BBQ Joint', 'Bakery', 'Bar', 'Beer Bar', 'Bike Shop', 'Bistro', 'Bookstore', 'Boutique', 'Breakfast Spot', 'Burger Joint', 'Burrito Place', 'Butcher', 'Café', 'Canal', 'Canal Lock', 'Candy Store', 'Caribbean Restaurant', 'Caucasian Restaurant', 'Cheese Shop', 'Chocolate Shop', 'Circus School', 'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'Comedy Club', 'Cosmetics Shop', 'Costume Shop', 'Coworking Space', 'Cupcake Shop', 'Dance Studio', 'Deli / Bodega', 'Donut Shop', 'Event Space', 'Farmers Market', 'Fast Food Restaurant', 'Fish & Chips Shop', 'Fish Market', 'Flea Market', 'Food & Drink Shop', 'Food Stand', 'Food Truck', 'French Restaurant', 'Fried Chicken Joint', 'Furniture / Home Store', 'Garden', 'Gastropub', 'Gift Shop', 'Grocery Store', 'Gym', 'Gym / Fitness C

In [97]:
#each neighborhood along with the top 10 most common venues

num_top_venues = 10

for hood in LDN_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = LDN_grouped[LDN_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Angel----
                      venue  freq
0                       Pub  0.09
1               Coffee Shop  0.05
2                      Café  0.05
3      Gym / Fitness Center  0.03
4             Burrito Place  0.03
5                Food Truck  0.03
6        Mexican Restaurant  0.03
7                Restaurant  0.03
8  Mediterranean Restaurant  0.03
9       Arts & Crafts Store  0.03


----Islington----
                      venue  freq
0                       Pub  0.12
1              Burger Joint  0.04
2  Mediterranean Restaurant  0.04
3                    Bakery  0.04
4                   Theater  0.03
5                Restaurant  0.03
6            Breakfast Spot  0.03
7       Japanese Restaurant  0.03
8                 Bookstore  0.03
9            Ice Cream Shop  0.03


----Notting Hill----
                venue  freq
0                 Pub  0.09
1  Italian Restaurant  0.07
2         Coffee Shop  0.05
3              Bakery  0.05
4      Clothing Store  0.05
5       Grocery Store  0.04

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = LDN_grouped['Neighborhood']

for ind in np.arange(LDN_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(LDN_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angel,Pub,Café,Coffee Shop,Food Truck,Arts & Crafts Store,Hotel,Indian Restaurant,Burrito Place,Mediterranean Restaurant,Mexican Restaurant
1,Islington,Pub,Mediterranean Restaurant,Bakery,Burger Joint,Bookstore,Theater,Japanese Restaurant,Breakfast Spot,Restaurant,Café
2,Notting Hill,Pub,Italian Restaurant,Bakery,Clothing Store,Coffee Shop,Cheese Shop,Deli / Bodega,Grocery Store,Restaurant,Hotel
3,Shoreditch,Coffee Shop,Italian Restaurant,Café,Hotel,Pub,Restaurant,Cocktail Bar,Gym / Fitness Center,Pizza Place,Sandwich Place
4,Spitalfields,Café,Flea Market,Indian Restaurant,Cocktail Bar,Burger Joint,Fish & Chips Shop,Pub,Bookstore,Pilates Studio,Park


In [100]:
#Clustering Neighborhoods
# set number of clusters
kclusters = 5

LDN_grouped_clustering = LDN_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(LDN_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([3, 2, 4, 1, 0], dtype=int32)

In [101]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels0', kmeans.labels_)

LDN_merged = LDN_ideal

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
LDN_merged = LDN_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

LDN_merged.head() # check the last columns!


,Neighborhood,Latitude,Longitude,Cluster Labels0,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angel,51.531946,-0.106106,3,Pub,Café,Coffee Shop,Food Truck,Arts & Crafts Store,Hotel,Indian Restaurant,Burrito Place,Mediterranean Restaurant,Mexican Restaurant
1,Islington,51.538429,-0.099905,2,Pub,Mediterranean Restaurant,Bakery,Burger Joint,Bookstore,Theater,Japanese Restaurant,Breakfast Spot,Restaurant,Café
2,Notting Hill,51.510999,-0.205527,4,Pub,Italian Restaurant,Bakery,Clothing Store,Coffee Shop,Cheese Shop,Deli / Bodega,Grocery Store,Restaurant,Hotel
3,Shoreditch,51.526669,-0.079893,1,Coffee Shop,Italian Restaurant,Café,Hotel,Pub,Restaurant,Cocktail Bar,Gym / Fitness Center,Pizza Place,Sandwich Place
4,Spitalfields,51.519527,-0.075170,0,Café,Flea Market,Indian Restaurant,Cocktail Bar,Burger Joint,Fish & Chips Shop,Pub,Bookstore,Pilates Studio,Park


In [102]:
#checking column names
print(LDN_merged.columns.tolist())

['Neighborhood', 'Latitude', 'Longitude', 'Cluster Labels0', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']


In [103]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LDN_merged['Latitude'], LDN_merged['Longitude'], LDN_merged['Neighborhood'], LDN_merged['Cluster Labels0']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [104]:
#examining clusters
LDN_0=LDN_merged.loc[LDN_merged['Cluster Labels0'] == 0, LDN_merged.columns[[1] + list(range(5, LDN_merged.shape[1]))]]
LDN_0

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,51.519527,Flea Market,Indian Restaurant,Cocktail Bar,Burger Joint,Fish & Chips Shop,Pub,Bookstore,Pilates Studio,Park


In [105]:
LDN_merged.loc[LDN_merged['Cluster Labels0'] == 1, LDN_merged.columns[[1] + list(range(5, LDN_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,51.526669,Italian Restaurant,Café,Hotel,Pub,Restaurant,Cocktail Bar,Gym / Fitness Center,Pizza Place,Sandwich Place


In [106]:
print(LDN_0.columns.tolist())

['Latitude', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']


In [107]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Angel, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'coffee'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


51.5319458 -0.1061056
coffee .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5319458,-0.1061056&v=20180604&query=coffee&radius=500&limit=100'

In [108]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f039009e116f90c6cf1bebf'},
 'response': {'venues': [{'id': '4d661f7ff75c3704343dd49a',
    'name': 'Wells Coffee Shop & Deli',
    'location': {'address': '397 St John St.',
     'lat': 51.53045276192664,
     'lng': -0.10617256164550781,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.53045276192664,
       'lng': -0.10617256164550781}],
     'distance': 166,
     'postalCode': 'EC1V 4LD',
     'cc': 'GB',
     'city': 'London',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['397 St John St.',
      'London',
      'Greater London',
      'EC1V 4LD',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594069147

In [109]:
venues = results['response']['venues']

# tranform venues into a dataframe
Angel_Coffee = json_normalize(venues)
Angel_Coffee.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4d661f7ff75c3704343dd49a,397 St John St.,GB,London,United Kingdom,NaN,166,"[397 St John St., London, Greater London, EC1V...","[{'label': 'display', 'lat': 51.53045276192664...",51.530453,-0.106173,EC1V 4LD,Greater London,Wells Coffee Shop & Deli,v-1594069147
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4e43f2cfa809d4ed1bb5c219,NaN,GB,NaN,United Kingdom,NaN,15,[United Kingdom],"[{'label': 'display', 'lat': 51.53193664550781...",51.531937,-0.105877,NaN,NaN,Augoato Coffee,v-1594069147
2,"[{'id': '4d4b7105d754a06378d81259', 'name': 'S...",False,4e4a8e891f6e2d88d8bad52e,NaN,GB,NaN,United Kingdom,NaN,26,[United Kingdom],"[{'label': 'display', 'lat': 51.53182983398437...",51.531830,-0.105765,NaN,NaN,Agusto Coffee,v-1594069147
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4eb10e889a52c49f4cd2e953,NaN,GB,NaN,United Kingdom,NaN,74,[United Kingdom],"[{'label': 'display', 'lat': 51.53138156992119...",51.531382,-0.106681,NaN,NaN,Jamie's Coffee Trike,v-1594069147
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5d4e9706d16e200008a1404f,barnsbury road,GB,London,United Kingdom,NaN,481,"[barnsbury road, London, Greater London, N1 0H...","[{'label': 'display', 'lat': 51.53459, 'lng': ...",51.534590,-0.111615,N1 0HG,Greater London,Coffee Berry,v-1594069147


In [110]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Angel_Coffee.columns if col.startswith('location.')] + ['id']
Angel_Coffee_filtered = Angel_Coffee.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Angel_Coffee_filtered['categories'] = Angel_Coffee_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Angel_Coffee_filtered.columns = [column.split('.')[-1] for column in Angel_Coffee_filtered.columns]

print(Angel_Coffee_filtered.count())
Angel_Coffee_filtered


name                15
categories          15
address             11
cc                  15
city                11
country             15
crossStreet          1
distance            15
formattedAddress    15
labeledLatLngs      15
lat                 15
lng                 15
postalCode           9
state               11
id                  15
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Wells Coffee Shop & Deli,Coffee Shop,397 St John St.,GB,London,United Kingdom,NaN,166,"[397 St John St., London, Greater London, EC1V...","[{'label': 'display', 'lat': 51.53045276192664...",51.530453,-0.106173,EC1V 4LD,Greater London,4d661f7ff75c3704343dd49a
1,Augoato Coffee,Café,NaN,GB,NaN,United Kingdom,NaN,15,[United Kingdom],"[{'label': 'display', 'lat': 51.53193664550781...",51.531937,-0.105877,NaN,NaN,4e43f2cfa809d4ed1bb5c219
2,Agusto Coffee,Shop & Service,NaN,GB,NaN,United Kingdom,NaN,26,[United Kingdom],"[{'label': 'display', 'lat': 51.53182983398437...",51.531830,-0.105765,NaN,NaN,4e4a8e891f6e2d88d8bad52e
3,Jamie's Coffee Trike,Coffee Shop,NaN,GB,NaN,United Kingdom,NaN,74,[United Kingdom],"[{'label': 'display', 'lat': 51.53138156992119...",51.531382,-0.106681,NaN,NaN,4eb10e889a52c49f4cd2e953
4,Coffee Berry,Coffee Shop,barnsbury road,GB,London,United Kingdom,NaN,481,"[barnsbury road, London, Greater London, N1 0H...","[{'label': 'display', 'lat': 51.53459, 'lng': ...",51.534590,-0.111615,N1 0HG,Greater London,5d4e9706d16e200008a1404f
5,Costa Coffee,Coffee Shop,32 Chapel Market,GB,Islington,United Kingdom,NaN,228,"[32 Chapel Market, Islington, Greater London, ...","[{'label': 'display', 'lat': 51.53362267230321...",51.533623,-0.108009,N1 9EN,Greater London,56deddd6cd108a565a2aac99
6,Scarlett Coffee Roastery,Coffee Shop,30 Duncan St,GB,London,United Kingdom,NaN,217,"[30 Duncan St, London, Greater London, N1 8BW,...","[{'label': 'display', 'lat': 51.53357, 'lng': ...",51.533570,-0.104370,N1 8BW,Greater London,5cd6d3c7da5ede002ce4d289
7,Penton Street Coffee,Coffee Shop,42 Penton Street,GB,London,United Kingdom,NaN,375,"[42 Penton Street, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.532948, 'lng':...",51.532948,-0.111285,N1 9QA,Greater London,5cefc2141af852002c350bb5
8,Starbucks,Coffee Shop,7 Islington High st,GB,London,United Kingdom,NaN,40,"[7 Islington High st, London, Greater London, ...","[{'label': 'display', 'lat': 51.532262, 'lng':...",51.532262,-0.106409,N1 9LQ,Greater London,4b5da672f964a520166529e3
9,Costa Coffee,Coffee Shop,Northampton Square,GB,London,United Kingdom,NaN,550,"[Northampton Square, London, Greater London, U...","[{'label': 'display', 'lat': 51.52762934526674...",51.527629,-0.102214,NaN,Greater London,5119034ee4b06e75ed988b0b


In [111]:
#visualize coffee
AngelCoffee_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(AngelCoffee_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(Angel_Coffee_filtered.lat, Angel_Coffee_filtered.lng, Angel_Coffee_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(AngelCoffee_map)

# display map
AngelCoffee_map

In [112]:
#explore top 5 coffee venues in Angel 
venue_id = '4d661f7ff75c3704343dd49a' # ID of Wells Coffee Shop & Deli
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url


'https://api.foursquare.com/v2/venues/4d661f7ff75c3704343dd49a?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&v=20180604'

In [113]:
#get venue details
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']


dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes'])


{'id': '4d661f7ff75c3704343dd49a',
 'name': 'Wells Coffee Shop & Deli',
 'contact': {},
 'location': {'address': '397 St John St.',
  'lat': 51.53045276192664,
  'lng': -0.10617256164550781,
  'labeledLatLngs': [{'label': 'display',
    'lat': 51.53045276192664,
    'lng': -0.10617256164550781}],
  'postalCode': 'EC1V 4LD',
  'cc': 'GB',
  'city': 'London',
  'state': 'Greater London',
  'country': 'United Kingdom',
  'formattedAddress': ['397 St John St.',
   'London',
   'Greater London',
   'EC1V 4LD',
   'United Kingdom']},
 'canonicalUrl': 'https://foursquare.com/v/wells-coffee-shop--deli/4d661f7ff75c3704343dd49a',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 5},
 'likes': {'count': 0, 'groups': []},
 'dislike': False,
 

In [114]:
#get venue overall rating
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [115]:
#get number of tips
result['response']['venue']['tips']['count']

5

In [116]:
venue_id = '4e43f2cfa809d4ed1bb5c219' #Augoato Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [117]:
result['response']['venue']['tips']['count']


0

In [118]:
venue_id = '4e4a8e891f6e2d88d8bad52e' #Agusto Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [119]:
result['response']['venue']['tips']['count']


0

In [120]:
venue_id = '4eb10e889a52c49f4cd2e953' #Jamie's Coffee Trike
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [121]:
result['response']['venue']['tips']['count']


1

In [122]:
venue_id = '5d4e9706d16e200008a1404f' #Coffee Berry
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [123]:
result['response']['venue']['tips']['count']


0

In [124]:
venue_id = '56deddd6cd108a565a2aac99' #Costa Coffee [index=5]
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.3


In [125]:
result['response']['venue']['tips']['count']


1

In [126]:
venue_id = '5cd6d3c7da5ede002ce4d289' #Scarlett Coffee Roastery 

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


In [127]:
result['response']['venue']['tips']['count']

3

In [128]:
venue_id = '4b5da672f964a520166529e3' #Starbucks
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.5


In [129]:
result['response']['venue']['tips']['count']

25

In [130]:
venue_id = '5cefc2141af852002c350bb5' #Penton Street Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [131]:
result['response']['venue']['tips']['count']

0

In [132]:
venue_id = '534dd311498e84b2e5a6546c' #passion noir coffee

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [133]:
venue_id = '4f046ac929c2b9a3edffef0d' #ground control

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.7


In [134]:
result['response']['venue']['tips']['count']

39

In [135]:
venue_id = '4fc9ff8ce4b087d229f75ce4' #The Coffeeworks Project	Coffee 


url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.2


In [136]:
result['response']['venue']['tips']['count']

140

In [137]:
#The Pump House Cafe
venue_id = '516fa20ce4b0ea41cb6744b8' 

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.1


In [138]:
result['response']['venue']['tips']['count']

7

In [139]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Islington, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'coffee'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5384287 -0.0999051
coffee .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5384287,-0.0999051&v=20180604&query=coffee&radius=500&limit=100'

In [140]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f039082f1677070ae94def6'},
 'response': {'venues': [{'id': '5e74c951646b990008eddfb6',
    'name': 'Devotion Coffee',
    'location': {'address': 'Cross Street',
     'crossStreet': 'Essex Road',
     'lat': 51.538805,
     'lng': -0.099041,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.538805,
       'lng': -0.099041}],
     'distance': 73,
     'postalCode': 'N1 2BL',
     'cc': 'GB',
     'city': 'London',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['Cross Street (Essex Road)',
      'London',
      'Greater London',
      'N1 2BL',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594069153',
    'has

In [141]:
venues = results['response']['venues']

# tranform venues into a dataframe
Isling_Coffee = json_normalize(venues)
Isling_Coffee.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5e74c951646b990008eddfb6,Cross Street,GB,London,United Kingdom,Essex Road,73,"[Cross Street (Essex Road), London, Greater Lo...","[{'label': 'display', 'lat': 51.538805, 'lng':...",51.538805,-0.099041,NaN,N1 2BL,Greater London,Devotion Coffee,v-1594069153
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4fb22a7de4b010d66eaca6d2,48 Essex Rd,GB,NaN,United Kingdom,NaN,88,"[48 Essex Rd, N1 8LR, United Kingdom]","[{'label': 'display', 'lat': 51.53763515212751...",51.537635,-0.100000,NaN,N1 8LR,NaN,Dolce Coffee,v-1594069153
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,50bb5f0ee4b069aaeb023fed,117-119 Essex Rd,GB,London,United Kingdom,NaN,143,"[117-119 Essex Rd, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.53919999373572...",51.539200,-0.098242,Canonbury,N1 2SN,Greater London,Costa Coffee,v-1594069153
3,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4c4408bb429a0f47a632481e,NaN,GB,NaN,United Kingdom,NaN,467,[United Kingdom],"[{'label': 'display', 'lat': 51.541223, 'lng':...",51.541223,-0.094874,NaN,NaN,NaN,Juleebucks Coffee House,v-1594069153
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5cd6d3c7da5ede002ce4d289,30 Duncan St,GB,London,United Kingdom,NaN,622,"[30 Duncan St, London, Greater London, N1 8BW,...","[{'label': 'display', 'lat': 51.53357, 'lng': ...",51.533570,-0.104370,NaN,N1 8BW,Greater London,Scarlett Coffee Roastery,v-1594069153


In [142]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Isling_Coffee.columns if col.startswith('location.')] + ['id']
Isling_Coffee_filtered = Isling_Coffee.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Isling_Coffee_filtered['categories'] = Isling_Coffee_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Isling_Coffee_filtered.columns = [column.split('.')[-1] for column in Isling_Coffee_filtered.columns]

print(Isling_Coffee_filtered.count())
Isling_Coffee_filtered

name                6
categories          6
address             5
cc                  6
city                4
country             6
crossStreet         1
distance            6
formattedAddress    6
labeledLatLngs      6
lat                 6
lng                 6
neighborhood        1
postalCode          5
state               4
id                  6
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Devotion Coffee,Coffee Shop,Cross Street,GB,London,United Kingdom,Essex Road,73,"[Cross Street (Essex Road), London, Greater Lo...","[{'label': 'display', 'lat': 51.538805, 'lng':...",51.538805,-0.099041,NaN,N1 2BL,Greater London,5e74c951646b990008eddfb6
1,Dolce Coffee,Coffee Shop,48 Essex Rd,GB,NaN,United Kingdom,NaN,88,"[48 Essex Rd, N1 8LR, United Kingdom]","[{'label': 'display', 'lat': 51.53763515212751...",51.537635,-0.100000,NaN,N1 8LR,NaN,4fb22a7de4b010d66eaca6d2
2,Costa Coffee,Coffee Shop,117-119 Essex Rd,GB,London,United Kingdom,NaN,143,"[117-119 Essex Rd, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.53919999373572...",51.539200,-0.098242,Canonbury,N1 2SN,Greater London,50bb5f0ee4b069aaeb023fed
3,Juleebucks Coffee House,Café,NaN,GB,NaN,United Kingdom,NaN,467,[United Kingdom],"[{'label': 'display', 'lat': 51.541223, 'lng':...",51.541223,-0.094874,NaN,NaN,NaN,4c4408bb429a0f47a632481e
4,Scarlett Coffee Roastery,Coffee Shop,30 Duncan St,GB,London,United Kingdom,NaN,622,"[30 Duncan St, London, Greater London, N1 8BW,...","[{'label': 'display', 'lat': 51.53357, 'lng': ...",51.533570,-0.104370,NaN,N1 8BW,Greater London,5cd6d3c7da5ede002ce4d289
5,The Coffeeworks Project,Coffee Shop,96-98 Islington High St,GB,Islington,United Kingdom,NaN,570,"[96-98 Islington High St, Islington, Greater L...","[{'label': 'display', 'lat': 51.53425428787325...",51.534254,-0.104684,NaN,N1 8EG,Greater London,4fc9ff8ce4b087d229f75ce4


In [143]:
#visualize coffee
IslingCoffee_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(IslingCoffee_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(Isling_Coffee_filtered.lat, Isling_Coffee_filtered.lng, Isling_Coffee_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(IslingCoffee_map)

# display map
IslingCoffee_map

In [144]:
#Devotion Coffee
#explore venue ratings
venue_id = '5e74c951646b990008eddfb6' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [145]:
#venue tips count
result['response']['venue']['tips']['count']

0

In [146]:
#Dolce Coffee
venue_id = '4fb22a7de4b010d66eaca6d2' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')



6.5


In [147]:
#venue tips count
result['response']['venue']['tips']['count']

3

In [148]:
#Costa Coffee
venue_id = '50bb5f0ee4b069aaeb023fed' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.2


In [149]:
#venue tips count
result['response']['venue']['tips']['count']

5

In [150]:
#Juleebucks Coffee House
venue_id = '4c4408bb429a0f47a632481e' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [151]:
#venue tips count
result['response']['venue']['tips']['count']

0

In [152]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Notting Hill, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'coffee'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5109995 -0.2055267
coffee .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5109995,-0.2055267&v=20180604&query=coffee&radius=500&limit=100'

In [153]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
Notting_Coffee = json_normalize(venues)
Notting_Coffee

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,5b26585dd7627e002cda85ea,53 Pembridge Rd,GB,London,United Kingdom,557,"[53 Pembridge Rd, London, Greater London, W11 ...","[{'label': 'display', 'lat': 51.51075285859169...",51.510753,-0.197494,Pembridge,W11 3HG,Greater London,Amoret Speciality Coffee,v-1594069155
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4b44e0bcf964a5200bff25e3,180 Portobello Rd,GB,London,United Kingdom,527,"[180 Portobello Rd, Notting Hill, Greater Lond...","[{'label': 'display', 'lat': 51.51571776745678...",51.515718,-0.204787,NaN,W11 2EB,Greater London,Coffee Plant,v-1594069155
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4acf8f02f964a52089d420e3,76 - 78 Holland Park Avenue,GB,London,United Kingdom,417,"[76 - 78 Holland Park Avenue, London, Greater ...","[{'label': 'display', 'lat': 51.507256, 'lng':...",51.507256,-0.205820,Kensington and Chelsea,W11 3RB,Greater London,Starbucks,v-1594069155
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4d64f88cd2906a310083e2cb,26 Pembridge Road,GB,London,United Kingdom,592,"[26 Pembridge Road, London, Greater London, W1...","[{'label': 'display', 'lat': 51.509785, 'lng':...",51.509785,-0.197193,NaN,W11 3HL,Greater London,Starbucks,v-1594069155
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4bb4ac06bc82a593f3a63e72,227 Portobello Road,GB,London,United Kingdom,649,"[227 Portobello Road, London, Greater London, ...","[{'label': 'display', 'lat': 51.516834, 'lng':...",51.516834,-0.205825,NaN,W11 1LT,Greater London,Starbucks,v-1594069155


In [154]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Notting_Coffee.columns if col.startswith('location.')] + ['id']
Notting_Coffee_filtered = Notting_Coffee.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Notting_Coffee_filtered['categories'] = Notting_Coffee_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Notting_Coffee_filtered.columns = [column.split('.')[-1] for column in Notting_Coffee_filtered.columns]

print(Angel_Coffee_filtered.count())
Notting_Coffee_filtered

name                15
categories          15
address             11
cc                  15
city                11
country             15
crossStreet          1
distance            15
formattedAddress    15
labeledLatLngs      15
lat                 15
lng                 15
postalCode           9
state               11
id                  15
dtype: int64


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Amoret Speciality Coffee,Coffee Shop,53 Pembridge Rd,GB,London,United Kingdom,557,"[53 Pembridge Rd, London, Greater London, W11 ...","[{'label': 'display', 'lat': 51.51075285859169...",51.510753,-0.197494,Pembridge,W11 3HG,Greater London,5b26585dd7627e002cda85ea
1,Coffee Plant,Coffee Shop,180 Portobello Rd,GB,London,United Kingdom,527,"[180 Portobello Rd, Notting Hill, Greater Lond...","[{'label': 'display', 'lat': 51.51571776745678...",51.515718,-0.204787,NaN,W11 2EB,Greater London,4b44e0bcf964a5200bff25e3
2,Starbucks,Coffee Shop,76 - 78 Holland Park Avenue,GB,London,United Kingdom,417,"[76 - 78 Holland Park Avenue, London, Greater ...","[{'label': 'display', 'lat': 51.507256, 'lng':...",51.507256,-0.205820,Kensington and Chelsea,W11 3RB,Greater London,4acf8f02f964a52089d420e3
3,Starbucks,Coffee Shop,26 Pembridge Road,GB,London,United Kingdom,592,"[26 Pembridge Road, London, Greater London, W1...","[{'label': 'display', 'lat': 51.509785, 'lng':...",51.509785,-0.197193,NaN,W11 3HL,Greater London,4d64f88cd2906a310083e2cb
4,Starbucks,Coffee Shop,227 Portobello Road,GB,London,United Kingdom,649,"[227 Portobello Road, London, Greater London, ...","[{'label': 'display', 'lat': 51.516834, 'lng':...",51.516834,-0.205825,NaN,W11 1LT,Greater London,4bb4ac06bc82a593f3a63e72


In [155]:
#visualize coffee
NottingCoffee_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(NottingCoffee_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(Notting_Coffee_filtered.lat, Notting_Coffee_filtered.lng, Notting_Coffee_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(NottingCoffee_map)

# display map
NottingCoffee_map

In [156]:
#explore venue ratings
venue_id = '5b26585dd7627e002cda85ea' #Amoret Speciality Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')



8.9


In [157]:
#venue tips count
result['response']['venue']['tips']['count']

11

In [158]:
#Coffee Plant  
#explore venue ratings
venue_id = '4b44e0bcf964a5200bff25e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.8


In [159]:
#venue tips count
result['response']['venue']['tips']['count']

67

In [160]:
#Starbucks
venue_id = '4acf8f02f964a52089d420e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


In [161]:
#venue tips count
result['response']['venue']['tips']['count']

10

In [162]:
#Starbucks
venue_id = '4d64f88cd2906a310083e2cb' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


In [163]:
#venue tips count
result['response']['venue']['tips']['count']

17

In [164]:
#Starbucks
venue_id = '4bb4ac06bc82a593f3a63e72' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.7


In [165]:
#venue tips count
result['response']['venue']['tips']['count']

11

In [166]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Shoreditch, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'coffee'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5266694 -0.0798926
coffee .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5266694,-0.0798926&v=20180604&query=coffee&radius=500&limit=100'

In [167]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f03905ca45aef3f3e693d91'},
 'response': {'venues': [{'id': '51de7169498e149ff9e5054e',
    'name': 'Andronicas World Of Coffee',
    'location': {'address': '91-93 Great Eastern Street',
     'lat': 51.526025143564034,
     'lng': -0.08343492278268182,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.526025143564034,
       'lng': -0.08343492278268182}],
     'distance': 255,
     'cc': 'GB',
     'country': 'United Kingdom',
     'formattedAddress': ['91-93 Great Eastern Street', 'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594069157',
    'hasPerk': False},
   {'id': '4fa8d80ce4b0f293ac00b41d',
    'name': "Silvio's Coffee House",
    'location': {'addre

In [168]:
venues = results['response']['venues']

# tranform venues into a dataframe
Shoreditch_Coffee = json_normalize(venues)
Shoreditch_Coffee.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,51de7169498e149ff9e5054e,91-93 Great Eastern Street,GB,NaN,United Kingdom,NaN,255,"[91-93 Great Eastern Street, United Kingdom]","[{'label': 'display', 'lat': 51.52602514356403...",51.526025,-0.083435,NaN,NaN,NaN,Andronicas World Of Coffee,v-1594069157,NaN
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4fa8d80ce4b0f293ac00b41d,6 Ravey St,GB,London,United Kingdom,NaN,239,"[6 Ravey St, London, Greater London, E C2A, Un...","[{'label': 'display', 'lat': 51.52500680640774...",51.525007,-0.082078,Hackney,E C2A,Greater London,Silvio's Coffee House,v-1594069157,NaN
2,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4d02110418ff8cfab971c257,134 Kingsland Rd.,GB,Hackney,United Kingdom,NaN,508,"[134 Kingsland Rd., Hackney, Greater London, E...","[{'label': 'display', 'lat': 51.53076674208519...",51.530767,-0.076633,NaN,E2 8DY,Greater London,Moretti Coffee Shop,v-1594069157,NaN
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4f54937ce4b0e14ed9512da2,11 Leonard St,GB,London,United Kingdom,NaN,522,"[11 Leonard St, London, Greater London, EC2A 4...","[{'label': 'display', 'lat': 51.52469267882992...",51.524693,-0.086737,"Shoreditch, London, Greater London",EC2A 4AQ,Greater London,Ozone Coffee Roasters,v-1594069157,186043995
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4f5f0740e4b0912b83bffdc4,"Units 8 & 9, 201 Bishopsgate Building",GB,London,United Kingdom,Norton Folgate,633,"[Units 8 & 9, 201 Bishopsgate Building (Norton...","[{'label': 'display', 'lat': 51.52101364246427...",51.521014,-0.078882,"Bishopsgate, London, Greater London",EC2M 3UG,Greater London,Department of Coffee and Social Affairs,v-1594069157,NaN


In [169]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Shoreditch_Coffee.columns if col.startswith('location.')] + ['id']
Shoreditch_Coffee_filtered = Shoreditch_Coffee.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Shoreditch_Coffee_filtered['categories'] = Shoreditch_Coffee_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Shoreditch_Coffee_filtered.columns = [column.split('.')[-1] for column in Shoreditch_Coffee_filtered.columns]

print(Shoreditch_Coffee_filtered.count())
Shoreditch_Coffee_filtered


name                39
categories          39
address             25
cc                  39
city                29
country             39
crossStreet          5
distance            39
formattedAddress    39
labeledLatLngs      39
lat                 39
lng                 39
neighborhood         9
postalCode          28
state               29
id                  39
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Andronicas World Of Coffee,Coffee Shop,91-93 Great Eastern Street,GB,NaN,United Kingdom,NaN,255,"[91-93 Great Eastern Street, United Kingdom]","[{'label': 'display', 'lat': 51.52602514356403...",51.526025,-0.083435,NaN,NaN,NaN,51de7169498e149ff9e5054e
1,Silvio's Coffee House,Coffee Shop,6 Ravey St,GB,London,United Kingdom,NaN,239,"[6 Ravey St, London, Greater London, E C2A, Un...","[{'label': 'display', 'lat': 51.52500680640774...",51.525007,-0.082078,Hackney,E C2A,Greater London,4fa8d80ce4b0f293ac00b41d
2,Moretti Coffee Shop,Café,134 Kingsland Rd.,GB,Hackney,United Kingdom,NaN,508,"[134 Kingsland Rd., Hackney, Greater London, E...","[{'label': 'display', 'lat': 51.53076674208519...",51.530767,-0.076633,NaN,E2 8DY,Greater London,4d02110418ff8cfab971c257
3,Ozone Coffee Roasters,Coffee Shop,11 Leonard St,GB,London,United Kingdom,NaN,522,"[11 Leonard St, London, Greater London, EC2A 4...","[{'label': 'display', 'lat': 51.52469267882992...",51.524693,-0.086737,"Shoreditch, London, Greater London",EC2A 4AQ,Greater London,4f54937ce4b0e14ed9512da2
4,Department of Coffee and Social Affairs,Coffee Shop,"Units 8 & 9, 201 Bishopsgate Building",GB,London,United Kingdom,Norton Folgate,633,"[Units 8 & 9, 201 Bishopsgate Building (Norton...","[{'label': 'display', 'lat': 51.52101364246427...",51.521014,-0.078882,"Bishopsgate, London, Greater London",EC2M 3UG,Greater London,4f5f0740e4b0912b83bffdc4
5,Black Opium Coffee,Coffee Shop,362-364 Old St,GB,London,United Kingdom,NaN,51,"[362-364 Old St, London, Greater London, EC1V ...","[{'label': 'display', 'lat': 51.52706057135338...",51.527061,-0.080287,NaN,EC1V 9LT,Greater London,589c3373ef787f18c12fbc5c
6,coffee bar falafel house,Falafel Restaurant,NaN,GB,London,United Kingdom,NaN,144,"[London, Greater London, EC1V 9LA, United King...","[{'label': 'display', 'lat': 51.526745, 'lng':...",51.526745,-0.081981,NaN,EC1V 9LA,Greater London,5c896130c0f163002cc4c81e
7,The Coffee Cup,Coffee Shop,NaN,GB,NaN,United Kingdom,NaN,143,[United Kingdom],"[{'label': 'display', 'lat': 51.52632992640448...",51.526330,-0.077901,NaN,NaN,NaN,57039aab498e1543a0a0b100
8,Coffee Studio,Coffee Shop,NaN,GB,London,United Kingdom,NaN,404,"[London, Greater London, E1, United Kingdom]","[{'label': 'display', 'lat': 51.523446, 'lng':...",51.523446,-0.077194,NaN,E1,Greater London,59632aceb3c9615804039ca3
9,Pretorius Coffee Bar,Coffee Shop,NaN,GB,NaN,United Kingdom,NaN,179,[United Kingdom],"[{'label': 'display', 'lat': 51.528032, 'lng':...",51.528032,-0.078500,NaN,NaN,NaN,514ad1f7e4b023bb96aba2dd


In [170]:
#visualize coffee
ShoreditchCoffee_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(ShoreditchCoffee_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(Shoreditch_Coffee_filtered.lat, Shoreditch_Coffee_filtered.lng, Shoreditch_Coffee_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(ShoreditchCoffee_map)

# display map
ShoreditchCoffee_map

In [172]:
#explore venue ratings
venue_id = '51de7169498e149ff9e5054e' #Andronicas World Of Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.9


In [173]:
result['response']['venue']['tips']['count']

2

In [177]:
#silvio's Coffee House
venue_id = '4fa8d80ce4b0f293ac00b41d' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.1


In [178]:
result['response']['venue']['tips']['count']

3

In [179]:
#Moretti Coffee Shop

venue_id = '4d02110418ff8cfab971c257' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [180]:
result['response']['venue']['tips']['count']

1

In [181]:
#Ozone Coffee Roasters
venue_id = '4f54937ce4b0e14ed9512da2' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.5


In [182]:
result['response']['venue']['tips']['count']

472

In [183]:
#Department of Coffee and Social Affairs
venue_id = '4f5f0740e4b0912b83bffdc4' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


In [184]:
result['response']['venue']['tips']['count']

62

In [185]:
#Black Opium Coffee
venue_id = '589c3373ef787f18c12fbc5c' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


In [186]:
result['response']['venue']['tips']['count']

6

In [187]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Spitalfields, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'coffee'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5195274 -0.0751701
coffee .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5195274,-0.0751701&v=20180604&query=coffee&radius=500&limit=100'

In [188]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f03968c521bc909eb4d58cb'},
 'response': {'venues': [{'id': '4d9722a3a2c65481086dd053',
    'name': 'Costa Coffee',
    'location': {'address': 'Unit 4-5, 250 Bishopsgate',
     'lat': 51.51922714304165,
     'lng': -0.07787535725867975,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.51922714304165,
       'lng': -0.07787535725867975}],
     'distance': 190,
     'postalCode': 'EC2M 4AA',
     'cc': 'GB',
     'city': 'Spitalfields',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['Unit 4-5, 250 Bishopsgate',
      'Spitalfields',
      'Greater London',
      'EC2M 4AA',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referr

In [190]:
venues = results['response']['venues']

# tranform venues into a dataframe
Spital_Coffee = json_normalize(venues)
print(Spital_Coffee.shape)
Spital_Coffee.head()

(50, 18)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4d9722a3a2c65481086dd053,"Unit 4-5, 250 Bishopsgate",GB,Spitalfields,United Kingdom,NaN,190,"[Unit 4-5, 250 Bishopsgate, Spitalfields, Grea...","[{'label': 'display', 'lat': 51.51922714304165...",51.519227,-0.077875,NaN,EC2M 4AA,Greater London,Costa Coffee,v-1594070800
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4c45f36c342c1b8dd12bb589,Commercial St,GB,London,United Kingdom,Fournier St,79,"[Commercial St (Fournier St), London, Greater ...","[{'label': 'display', 'lat': 51.5190712320545,...",51.519071,-0.074285,NaN,E1 6LY,Greater London,The Coffee Cup,v-1594070800
2,"[{'id': '52e81612bcbc57f1066b79ed', 'name': 'O...",False,5ac65b64005ac1229663a9fc,NaN,GB,London,United Kingdom,NaN,208,"[London, Greater London, E1 6EG, United Kingdom]","[{'label': 'display', 'lat': 51.519516, 'lng':...",51.519516,-0.078184,NaN,E1 6EG,Greater London,Dogman and Rabbitgirl with Coffee Statue,v-1594070800
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,58d69a399ab66329a840632c,25 Hanbury St,GB,London,United Kingdom,NaN,197,"[25 Hanbury St, London, Greater London, E1 6QR...","[{'label': 'display', 'lat': 51.520367, 'lng':...",51.520367,-0.072658,NaN,E1 6QR,Greater London,Nude Coffee Roasters Roastery,v-1594070800
4,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,5ae471d5f5e9d70039610765,Dray Walk,GB,London,United Kingdom,NaN,317,"[Dray Walk, London, Greater London, E1 6QL, Un...","[{'label': 'display', 'lat': 51.521328, 'lng':...",51.521328,-0.071622,NaN,E1 6QL,Greater London,Long & Short Coffee,v-1594070800


In [191]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Spital_Coffee.columns if col.startswith('location.')] + ['id']
Spital_Coffee_filtered = Spital_Coffee.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Spital_Coffee_filtered['categories'] = Spital_Coffee_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Spital_Coffee_filtered.columns = [column.split('.')[-1] for column in Spital_Coffee_filtered.columns]

print(Spital_Coffee_filtered.count())
Spital_Coffee_filtered

name                50
categories          50
address             35
cc                  50
city                40
country             50
crossStreet          6
distance            50
formattedAddress    50
labeledLatLngs      50
lat                 50
lng                 50
neighborhood         7
postalCode          36
state               40
id                  50
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Costa Coffee,Coffee Shop,"Unit 4-5, 250 Bishopsgate",GB,Spitalfields,United Kingdom,NaN,190,"[Unit 4-5, 250 Bishopsgate, Spitalfields, Grea...","[{'label': 'display', 'lat': 51.51922714304165...",51.519227,-0.077875,NaN,EC2M 4AA,Greater London,4d9722a3a2c65481086dd053
1,The Coffee Cup,Coffee Shop,Commercial St,GB,London,United Kingdom,Fournier St,79,"[Commercial St (Fournier St), London, Greater ...","[{'label': 'display', 'lat': 51.5190712320545,...",51.519071,-0.074285,NaN,E1 6LY,Greater London,4c45f36c342c1b8dd12bb589
2,Dogman and Rabbitgirl with Coffee Statue,Outdoor Sculpture,NaN,GB,London,United Kingdom,NaN,208,"[London, Greater London, E1 6EG, United Kingdom]","[{'label': 'display', 'lat': 51.519516, 'lng':...",51.519516,-0.078184,NaN,E1 6EG,Greater London,5ac65b64005ac1229663a9fc
3,Nude Coffee Roasters Roastery,Coffee Shop,25 Hanbury St,GB,London,United Kingdom,NaN,197,"[25 Hanbury St, London, Greater London, E1 6QR...","[{'label': 'display', 'lat': 51.520367, 'lng':...",51.520367,-0.072658,NaN,E1 6QR,Greater London,58d69a399ab66329a840632c
4,Long & Short Coffee,Café,Dray Walk,GB,London,United Kingdom,NaN,317,"[Dray Walk, London, Greater London, E1 6QL, Un...","[{'label': 'display', 'lat': 51.521328, 'lng':...",51.521328,-0.071622,NaN,E1 6QL,Greater London,5ae471d5f5e9d70039610765
5,Department of Coffee and Social Affairs,Coffee Shop,"Units 8 & 9, 201 Bishopsgate Building",GB,London,United Kingdom,Norton Folgate,305,"[Units 8 & 9, 201 Bishopsgate Building (Norton...","[{'label': 'display', 'lat': 51.52101364246427...",51.521014,-0.078882,"Bishopsgate, London, Greater London",EC2M 3UG,Greater London,4f5f0740e4b0912b83bffdc4
6,Costa Coffee,Coffee Shop,Liverpool Street Station,GB,London,United Kingdom,NaN,509,"[Liverpool Street Station, London, Greater Lon...","[{'label': 'display', 'lat': 51.5179911384162,...",51.517991,-0.082097,NaN,NaN,Greater London,4f7d577de4b0015bec70e7a0
7,AMT Coffee,Coffee Shop,Liverpool Street Railway Station,GB,London,United Kingdom,NaN,508,"[Liverpool Street Railway Station, London, Gre...","[{'label': 'display', 'lat': 51.51801431442288...",51.518014,-0.082097,NaN,EC2,Greater London,4bbba38bed7776b0b4b53e51
8,Mr Coffee & Mrs Cake,Food Truck,NaN,GB,London,United Kingdom,NaN,18,"[London, Greater London, E1 6BG, United Kingdom]","[{'label': 'display', 'lat': 51.519678, 'lng':...",51.519678,-0.075294,NaN,E1 6BG,Greater London,5b1411371de765002d694ba7
9,Morson Coffee stand at Spitalfields,Café,NaN,GB,NaN,United Kingdom,NaN,87,[United Kingdom],"[{'label': 'display', 'lat': 51.51888370513916...",51.518884,-0.074446,NaN,NaN,NaN,4be91865c5220f47a875aaca


In [192]:
#visualize coffee
SpitalCoffee_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(SpitalCoffee_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(Spital_Coffee_filtered.lat, Spital_Coffee_filtered.lng, Spital_Coffee_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(SpitalCoffee_map)

# display map
SpitalCoffee_map

In [216]:
#explore venue ratings
#gone through top 15 venues from spitalfields
venue_id = '58d7cf7eda54ae60aad4698b' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [194]:
#explore venue ratings
venue_id = '4c45f36c342c1b8dd12bb589' #the Coffee Cup
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [195]:
#explore venue ratings
venue_id = '5ac65b64005ac1229663a9fc' #Dogman
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [196]:
#explore venue ratings
venue_id = '58d69a399ab66329a840632c' #Nude Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [200]:
#explore venue ratings
venue_id = '5ae471d5f5e9d70039610765' #Long and short Coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Exploring nearby Grocery stores

In [233]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Notting Hill, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'grocery store'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5109995 -0.2055267
grocery store .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5109995,-0.2055267&v=20180604&query=grocery store&radius=500&limit=100'

In [234]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f039b925947492e3e34c563'},
 'response': {'venues': [{'id': '5681c673498e260887c1bcc2',
    'name': 'Premier convenience store',
    'location': {'address': '109 Clarendon Road',
     'lat': 51.512088,
     'lng': -0.210916,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.512088,
       'lng': -0.210916}],
     'distance': 392,
     'cc': 'GB',
     'neighborhood': 'Kensington and Chelsea',
     'city': 'London',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['109 Clarendon Road',
      'London',
      'Greater London',
      'United Kingdom']},
    'categories': [{'id': '4d954b0ea243a5684a65b473',
      'name': 'Convenience Store',
      'pluralName': 'Convenience Stores',
      'shortName': 'Convenience Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/conveniencestore_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594072575',
    '

In [235]:
venues = results['response']['venues']

# tranform venues into a dataframe
Notting_Groc = json_normalize(venues)
print(Notting_Groc.shape)
Notting_Groc.head()

(7, 17)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,5681c673498e260887c1bcc2,109 Clarendon Road,GB,London,United Kingdom,392,"[109 Clarendon Road, London, Greater London, U...","[{'label': 'display', 'lat': 51.512088, 'lng':...",51.512088,-0.210916,Kensington and Chelsea,NaN,Greater London,Premier convenience store,v-1594072575
1,"[{'id': '4bf58dd8d48988d108951735', 'name': 'W...",False,4ee654de6c25be96322f26e4,199 Portobello Road,GB,London,United Kingdom,549,"[199 Portobello Road, London, Greater London, ...","[{'label': 'display', 'lat': 51.51592500578841...",51.515925,-0.205030,NaN,NaN,Greater London,Gatheting goddes pop up vintage store,v-1594072575
2,"[{'id': '4bf58dd8d48988d107951735', 'name': 'S...",False,4e184ee5b0fb8567c66ae30a,187 Portobello Road,GB,London,United Kingdom,454,"[187 Portobello Road, London, Greater London, ...","[{'label': 'display', 'lat': 51.51496, 'lng': ...",51.514960,-0.207140,NaN,NaN,Greater London,The Leather & Boot Store,v-1594072575
3,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",False,4ea57d5429c2abc8e00daa77,2 Ladbroke Grove,GB,London,United Kingdom,351,"[2 Ladbroke Grove, London, Greater London, W11...","[{'label': 'display', 'lat': 51.50817244916098...",51.508172,-0.203264,Notting Hill,W11 3BG,Greater London,American Food Store,v-1594072575
4,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",False,4c9e0833031337044d6d64d5,105 portobello rd,GB,Kensington,United Kingdom,337,"[105 portobello rd, Kensington, Greater London...","[{'label': 'display', 'lat': 51.51323421100397...",51.513234,-0.202237,NaN,NaN,Greater London,Highland Store,v-1594072575


In [236]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Shoreditch, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'grocery store'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5266694 -0.0798926
grocery store .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5266694,-0.0798926&v=20180604&query=grocery store&radius=500&limit=100'

In [238]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f03a5af39be6735845d5355'},
 'response': {'venues': [{'id': '4ad30c88f964a52003e320e3',
    'name': 'The Grocery',
    'location': {'address': '54-56 Kingsland Rd',
     'lat': 51.52887127007046,
     'lng': -0.07779734768381653,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.52887127007046,
       'lng': -0.07779734768381653}],
     'distance': 284,
     'postalCode': 'E2 8DP',
     'cc': 'GB',
     'neighborhood': 'Hackney',
     'city': 'London',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['54-56 Kingsland Rd',
      'London',
      'Greater London',
      'E2 8DP',
      'United Kingdom']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b45',
      'name': 'Organic Grocery',
      'pluralName': 'Organic Groceries',
      'shortName': 'Organic Grocery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': Tr

In [239]:
venues = results['response']['venues']

# tranform venues into a dataframe
Shoreditch_Groc = json_normalize(venues)
print(Shoreditch_Groc.shape)
Shoreditch_Groc.head()

(41, 19)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52f2ab2ebcbc57f1066b8b45', 'name': 'O...",False,4ad30c88f964a52003e320e3,54-56 Kingsland Rd,GB,London,United Kingdom,NaN,284,"[54-56 Kingsland Rd, London, Greater London, E...","[{'label': 'display', 'lat': 51.52887127007046...",51.528871,-0.077797,Hackney,E2 8DP,Greater London,The Grocery,v-1594074678,NaN
1,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",False,513b0c9ce4b099af37f9ad5b,54-56 Kingsland Road,GB,London,United Kingdom,NaN,289,"[54-56 Kingsland Road, London, Greater London,...","[{'label': 'display', 'lat': 51.52891765636797...",51.528918,-0.077782,NaN,E2 8DP,Greater London,The Grocery Kitchen,v-1594074678,NaN
2,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,566c18da498ea7697febe43c,52 Kingsland Rd,GB,London,United Kingdom,NaN,280,"[52 Kingsland Rd, London, Greater London, E2 8...","[{'label': 'display', 'lat': 51.52882539882658...",51.528825,-0.077787,"Hackney, London, Greater London",E2 8DP,Greater London,The Grocery Cafe,v-1594074678,NaN
3,"[{'id': '52f2ab2ebcbc57f1066b8b45', 'name': 'O...",False,594ff11d3731ee3b68473412,NaN,GB,London,United Kingdom,NaN,99,"[London, Greater London, EC1V 9LL, United King...","[{'label': 'display', 'lat': 51.52729218171551...",51.527292,-0.078859,NaN,EC1V 9LL,Greater London,Raw Store,v-1594074678,NaN
4,[],False,4ca7236c44a8224b03eb0940,NaN,GB,NaN,United Kingdom,NaN,87,[United Kingdom],"[{'label': 'display', 'lat': 51.525903, 'lng':...",51.525903,-0.079636,NaN,NaN,NaN,The Store Rooms,v-1594074678,NaN


In [3]:
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [5]:
# Get Foursquare credentials
CLIENT_ID = 'CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3' # your Foursquare ID
CLIENT_SECRET = 'OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3
CLIENT_SECRET:OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0


In [29]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Shoreditch, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'market'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5266694 -0.0798926
market .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5266694,-0.0798926&v=20180604&query=market&radius=500&limit=30'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f04cfb28005581f724350b8'},
 'response': {'venues': [{'id': '4b643b76f964a52078a52ae3',
    'name': 'Hoxton Street Market',
    'location': {'address': 'Hoxton St',
     'lat': 51.532208093655534,
     'lng': -0.08017659187316895,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.532208093655534,
       'lng': -0.08017659187316895}],
     'distance': 616,
     'postalCode': 'N1',
     'cc': 'GB',
     'city': 'Hoxton',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['Hoxton St',
      'Hoxton',
      'Greater London',
      'N1',
      'United Kingdom']},
    'categories': [{'id': '50be8ee891d4fa8dcc7199a7',
      'name': 'Market',
      'pluralName': 'Markets',
      'shortName': 'Market',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/market_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594151385',
    'hasPerk': False},
   {'id': '4f465a6

In [31]:
venues = results['response']['venues']

# tranform venues into a dataframe
Shore_Market = json_normalize(venues)
print(Shore_Market.shape)
Shore_Market

(30, 19)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",False,4b643b76f964a52078a52ae3,Hoxton St,GB,Hoxton,United Kingdom,NaN,616,"[Hoxton St, Hoxton, Greater London, N1, United...","[{'label': 'display', 'lat': 51.53220809365553...",51.532208,-0.080177,NaN,N1,Greater London,Hoxton Street Market,v-1594151385,NaN
1,"[{'id': '4bf58dd8d48988d105941735', 'name': 'G...",False,4f465a6be4b0190ed36fca32,1-6 batemans row,GB,London,United Kingdom,NaN,191,"[1-6 batemans row, London, Greater London, EC2...","[{'label': 'display', 'lat': 51.52497298254310...",51.524973,-0.080346,NaN,EC2A 3HH,Greater London,Market Sports Swimming Pool,v-1594151385,NaN
2,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,52543fc211d2d424305c8d83,NaN,GB,NaN,United Kingdom,NaN,308,[United Kingdom],"[{'label': 'display', 'lat': 51.526505, 'lng':...",51.526505,-0.084344,NaN,NaN,NaN,Market Kitchen,v-1594151385,NaN
3,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4c8e14da58668cfa889bceec,NaN,GB,NaN,United Kingdom,NaN,320,[United Kingdom],"[{'label': 'display', 'lat': 51.524723, 'lng':...",51.524723,-0.083298,NaN,NaN,NaN,The Market,v-1594151385,NaN
4,"[{'id': '503287a291d4c4b30a586d65', 'name': 'F...",False,5da97b75a8303600079beb1d,"3rd Floor, 86-90 Paul Street",GB,London,United Kingdom,NaN,282,"[3rd Floor, 86-90 Paul Street, London, Greater...","[{'label': 'display', 'lat': 51.52567164269103...",51.525672,-0.083639,NaN,EC2A 4NE,Greater London,Developer Money Market,v-1594151385,NaN
5,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,4ff2eb53e4b01870b3d20f8c,NaN,GB,NaN,United Kingdom,NaN,327,[United Kingdom],"[{'label': 'display', 'lat': 51.52469865324323...",51.524699,-0.083406,NaN,NaN,NaN,Mini Market,v-1594151385,NaN
6,"[{'id': '4bf58dd8d48988d1f1931735', 'name': 'G...",False,540db905498ec23df3daa73e,73 Leonard Street,GB,London,United Kingdom,NaN,304,"[73 Leonard Street, London, Greater London, EC...","[{'label': 'display', 'lat': 51.52476582324468...",51.524766,-0.083041,NaN,EC2A 4QS,Greater London,The Saturday Market Project,v-1594151385,NaN
7,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",False,53d24447498e980f1ab9ed82,NaN,GB,NaN,United Kingdom,NaN,327,[United Kingdom],"[{'label': 'display', 'lat': 51.52625343883755...",51.526253,-0.084576,NaN,NaN,NaN,Old Street Food Market,v-1594151385,NaN
8,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,5e46a2e35105fb000882644d,Mark St,GB,NaN,United Kingdom,Mark Sq,635,"[Mark St (Mark Sq), United Kingdom]","[{'label': 'display', 'lat': 51.52308909730698...",51.523089,-0.087040,NaN,NaN,NaN,Mark Sq Food Market,v-1594151385,NaN
9,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",False,5c5ecc59d807ee002c8e2da2,NaN,GB,London,United Kingdom,NaN,491,"[London, Greater London, N1 6JF, United Kingdom]","[{'label': 'display', 'lat': 51.531027, 'lng':...",51.531027,-0.081043,NaN,N1 6JF,Greater London,Crondall Street Hoxton Street Market,v-1594151385,NaN


In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Shore_Market.columns if col.startswith('location.')] + ['id']
Shore_Market_filtered = Shore_Market.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Shore_Market_filtered['categories'] = Shore_Market_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Shore_Market_filtered.columns = [column.split('.')[-1] for column in Shore_Market_filtered.columns]

print(Shore_Market_filtered.count())
Shore_Market_filtered

name                30
categories          30
address             23
cc                  30
city                22
country             30
crossStreet          4
distance            30
formattedAddress    30
labeledLatLngs      30
lat                 30
lng                 30
neighborhood         1
postalCode          22
state               22
id                  30
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hoxton Street Market,Market,Hoxton St,GB,Hoxton,United Kingdom,NaN,616,"[Hoxton St, Hoxton, Greater London, N1, United...","[{'label': 'display', 'lat': 51.53220809365553...",51.532208,-0.080177,NaN,N1,Greater London,4b643b76f964a52078a52ae3
1,Market Sports Swimming Pool,Gym Pool,1-6 batemans row,GB,London,United Kingdom,NaN,191,"[1-6 batemans row, London, Greater London, EC2...","[{'label': 'display', 'lat': 51.52497298254310...",51.524973,-0.080346,NaN,EC2A 3HH,Greater London,4f465a6be4b0190ed36fca32
2,Market Kitchen,Vietnamese Restaurant,NaN,GB,NaN,United Kingdom,NaN,308,[United Kingdom],"[{'label': 'display', 'lat': 51.526505, 'lng':...",51.526505,-0.084344,NaN,NaN,NaN,52543fc211d2d424305c8d83
3,The Market,Office,NaN,GB,NaN,United Kingdom,NaN,320,[United Kingdom],"[{'label': 'display', 'lat': 51.524723, 'lng':...",51.524723,-0.083298,NaN,NaN,NaN,4c8e14da58668cfa889bceec
4,Developer Money Market,Financial or Legal Service,"3rd Floor, 86-90 Paul Street",GB,London,United Kingdom,NaN,282,"[3rd Floor, 86-90 Paul Street, London, Greater...","[{'label': 'display', 'lat': 51.52567164269103...",51.525672,-0.083639,NaN,EC2A 4NE,Greater London,5da97b75a8303600079beb1d
5,Mini Market,Convenience Store,NaN,GB,NaN,United Kingdom,NaN,327,[United Kingdom],"[{'label': 'display', 'lat': 51.52469865324323...",51.524699,-0.083406,NaN,NaN,NaN,4ff2eb53e4b01870b3d20f8c
6,The Saturday Market Project,General Entertainment,73 Leonard Street,GB,London,United Kingdom,NaN,304,"[73 Leonard Street, London, Greater London, EC...","[{'label': 'display', 'lat': 51.52476582324468...",51.524766,-0.083041,NaN,EC2A 4QS,Greater London,540db905498ec23df3daa73e
7,Old Street Food Market,Market,NaN,GB,NaN,United Kingdom,NaN,327,[United Kingdom],"[{'label': 'display', 'lat': 51.52625343883755...",51.526253,-0.084576,NaN,NaN,NaN,53d24447498e980f1ab9ed82
8,Mark Sq Food Market,Food Truck,Mark St,GB,NaN,United Kingdom,Mark Sq,635,"[Mark St (Mark Sq), United Kingdom]","[{'label': 'display', 'lat': 51.52308909730698...",51.523089,-0.087040,NaN,NaN,NaN,5e46a2e35105fb000882644d
9,Crondall Street Hoxton Street Market,Bus Stop,NaN,GB,London,United Kingdom,NaN,491,"[London, Greater London, N1 6JF, United Kingdom]","[{'label': 'display', 'lat': 51.531027, 'lng':...",51.531027,-0.081043,NaN,N1 6JF,Greater London,5c5ecc59d807ee002c8e2da2


In [11]:
#further exploring different categories: Coffee shops, cafe, grocery store, gyms, chinese restuarant
address = 'Spitalfields, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'grocery store'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5195274 -0.0751701
grocery store .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5195274,-0.0751701&v=20180604&query=grocery store&radius=500&limit=30'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f04ca7228796f3c9c2c7e83'},
 'response': {'venues': [{'id': '570a7d82498e0a570239b054',
    'name': 'Side Party Pop-Up Store',
    'location': {'lat': 51.520149,
     'lng': -0.074846,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.520149,
       'lng': -0.074846}],
     'distance': 72,
     'cc': 'GB',
     'country': 'United Kingdom',
     'formattedAddress': ['United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d108951735',
      'name': "Women's Store",
      'pluralName': "Women's Stores",
      'shortName': "Women's Store",
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_women_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594149591',
    'hasPerk': False},
   {'id': '4bfd4dc88992a5930ee5abb0',
    'name': 'Kings Stores',
    'location': {'address': '14 Widegate St',
     'lat': 51.51806773342955,
     'lng': -0.0778380633172337,
     'labeledLatLngs': [{'label'

In [15]:
venues = results['response']['venues']

# tranform venues into a dataframe
Spital_Groc = json_normalize(venues)
Spital_Groc.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d108951735', 'name': 'W...",False,570a7d82498e0a570239b054,NaN,GB,NaN,United Kingdom,NaN,72,[United Kingdom],"[{'label': 'display', 'lat': 51.520149, 'lng':...",51.520149,-0.074846,NaN,NaN,NaN,Side Party Pop-Up Store,v-1594149591,NaN
1,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",False,4bfd4dc88992a5930ee5abb0,14 Widegate St,GB,London,United Kingdom,NaN,246,"[14 Widegate St, London, Greater London, E1 7H...","[{'label': 'display', 'lat': 51.51806773342955...",51.518068,-0.077838,NaN,E1 7HP,Greater London,Kings Stores,v-1594149591,136172022
2,"[{'id': '4bf58dd8d48988d104951735', 'name': 'B...",False,5087dadbe4b0661f48b5ba76,12 Dray Walk,GB,London,United Kingdom,NaN,214,"[12 Dray Walk, London, Greater London, United ...","[{'label': 'display', 'lat': 51.52070081951377...",51.520701,-0.072716,NaN,NaN,Greater London,Obey Pop-up Store,v-1594149591,NaN
3,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",False,53c2892a498eaf310d969ff4,NaN,GB,NaN,United Kingdom,NaN,192,[United Kingdom],"[{'label': 'display', 'lat': 51.52072145102843...",51.520721,-0.073158,NaN,NaN,NaN,designer popup store,v-1594149591,NaN
4,"[{'id': '4bf58dd8d48988d1fe941735', 'name': 'M...",False,5363a1e0498e51c1bf7a1e30,NaN,GB,NaN,United Kingdom,NaN,361,[United Kingdom],"[{'label': 'display', 'lat': 51.52103514753769...",51.521035,-0.070544,NaN,NaN,NaN,Defected pop-up store,v-1594149591,NaN


In [16]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Spital_Groc.columns if col.startswith('location.')] + ['id']
Spital_Groc_filtered = Spital_Groc.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Spital_Groc_filtered['categories'] = Spital_Groc_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Spital_Groc_filtered.columns = [column.split('.')[-1] for column in Spital_Groc_filtered.columns]

print(Spital_Groc_filtered.count())
Spital_Groc_filtered

name                28
categories          28
address             17
cc                  28
city                16
country             28
crossStreet          3
distance            28
formattedAddress    28
labeledLatLngs      28
lat                 28
lng                 28
neighborhood         2
postalCode          11
state               16
id                  28
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Side Party Pop-Up Store,Women's Store,NaN,GB,NaN,United Kingdom,NaN,72,[United Kingdom],"[{'label': 'display', 'lat': 51.520149, 'lng':...",51.520149,-0.074846,NaN,NaN,NaN,570a7d82498e0a570239b054
1,Kings Stores,Pub,14 Widegate St,GB,London,United Kingdom,NaN,246,"[14 Widegate St, London, Greater London, E1 7H...","[{'label': 'display', 'lat': 51.51806773342955...",51.518068,-0.077838,NaN,E1 7HP,Greater London,4bfd4dc88992a5930ee5abb0
2,Obey Pop-up Store,Boutique,12 Dray Walk,GB,London,United Kingdom,NaN,214,"[12 Dray Walk, London, Greater London, United ...","[{'label': 'display', 'lat': 51.52070081951377...",51.520701,-0.072716,NaN,NaN,Greater London,5087dadbe4b0661f48b5ba76
3,designer popup store,Clothing Store,NaN,GB,NaN,United Kingdom,NaN,192,[United Kingdom],"[{'label': 'display', 'lat': 51.52072145102843...",51.520721,-0.073158,NaN,NaN,NaN,53c2892a498eaf310d969ff4
4,Defected pop-up store,Music Store,NaN,GB,NaN,United Kingdom,NaN,361,[United Kingdom],"[{'label': 'display', 'lat': 51.52103514753769...",51.521035,-0.070544,NaN,NaN,NaN,5363a1e0498e51c1bf7a1e30
5,JB Food Store,Food & Drink Shop,97 brick lane,GB,Poplar,United Kingdom,Quaker street,377,"[97 brick lane (Quaker street), Poplar, Greate...","[{'label': 'display', 'lat': 51.522134, 'lng':...",51.522134,-0.071690,NaN,NaN,Greater London,4b755325f964a5205c062ee3
6,Sylhet Super Store,Smoke Shop,NaN,GB,NaN,United Kingdom,NaN,415,[United Kingdom],"[{'label': 'display', 'lat': 51.51999508864033...",51.519995,-0.069220,NaN,NaN,NaN,5231865b11d284c45f704888
7,London Marathon Store,Sporting Goods Shop,1-3 Norton Folgate,GB,NaN,United Kingdom,NaN,283,"[1-3 Norton Folgate, E1 6DB, United Kingdom]","[{'label': 'display', 'lat': 51.52053131625524...",51.520531,-0.078930,NaN,E1 6DB,NaN,52d161ac498e92b3fc70e64b
8,Hospital Pop-up Store,Boutique,NaN,GB,NaN,United Kingdom,NaN,419,[United Kingdom],"[{'label': 'display', 'lat': 51.52315139770508...",51.523151,-0.076844,NaN,NaN,NaN,51c5d9b4498e7b1ee6902d36
9,IQOS Store,Smoke Shop,"Box No. 20, Boxpark Shoreditch, 2-10 Bethnal G...",GB,London,United Kingdom,NaN,450,"[Box No. 20, Boxpark Shoreditch, 2-10 Bethnal ...","[{'label': 'display', 'lat': 51.523499, 'lng':...",51.523499,-0.076456,NaN,E1 6GY,Greater London,5d88cdccd7f5300008eeb9f3


## querying markets

In [17]:
address = 'Angel, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'market'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5319458 -0.1061056
market .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5319458,-0.1061056&v=20180604&query=market&radius=500&limit=30'

In [19]:
results = requests.get(url).json()

venues = results['response']['venues']

# tranform venues into a dataframe
Angel_market = json_normalize(venues)
print(Angel_market.shape)
Angel_market

(18, 18)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '53e0feef498e5aac066fd8a9', 'name': 'S...",False,4ac518eef964a5201ead20e3,Chapel Market,GB,Islington,United Kingdom,at Liverpool Rd.,236,"[Chapel Market (at Liverpool Rd.), Islington, ...","[{'label': 'display', 'lat': 51.53352287954976...",51.533523,-0.108404,NaN,N1 9QA,Greater London,Chapel Market,v-1594150008
1,"[{'id': '4bf58dd8d48988d1fa941735', 'name': 'F...",False,4c0b6c36bbc676b0828c4bd5,Chapel Market,GB,Islington,United Kingdom,Penton St.,326,"[Chapel Market (Penton St.), Islington, Greate...","[{'label': 'display', 'lat': 51.53342836198379...",51.533428,-0.110168,NaN,N1 9PZ,Greater London,Islington Farmers’ Market,v-1594150008
2,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,4bffb0f0daf9c9b6c886faef,Exmouth Market,GB,Islington,United Kingdom,NaN,75,"[Exmouth Market, Islington, Greater London, Un...","[{'label': 'display', 'lat': 51.53146203403172...",51.531462,-0.106877,NaN,NaN,Greater London,Moro Market Stall,v-1594150008
3,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",False,4be90afac5220f47606daaca,NaN,GB,NaN,United Kingdom,NaN,410,[United Kingdom],"[{'label': 'display', 'lat': 51.53336226940155...",51.533362,-0.111580,NaN,NaN,NaN,Market News,v-1594150008
4,"[{'id': '4bf58dd8d48988d1f7941735', 'name': 'F...",False,50829f24e4b07dd17bd5da22,NaN,GB,NaN,United Kingdom,NaN,416,[United Kingdom],"[{'label': 'display', 'lat': 51.535317, 'lng':...",51.535317,-0.103482,NaN,NaN,NaN,Islington Antiques Market,v-1594150008
5,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",False,51d9231be4b00508f5b92f1a,Baron Street,GB,London,United Kingdom,NaN,213,"[Baron Street, London, Greater London, United ...","[{'label': 'display', 'lat': 51.532243, 'lng':...",51.532243,-0.109150,NaN,NaN,Greater London,Bus Stop T @ Baron Street (Chapel Market),v-1594150008
6,"[{'id': '4bf58dd8d48988d11d951735', 'name': 'B...",False,4f9c8457e4b0609168f221d9,35 Chapel Market,GB,London,United Kingdom,NaN,214,"[35 Chapel Market, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.5335461026408,...",51.533546,-0.107833,NaN,N1 9EN,Greater London,Fresh Meat Market,v-1594150008
7,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,4daae7d64df01c19b193a4cc,Chapel Market,GB,Islington,United Kingdom,NaN,227,"[Chapel Market, Islington, Greater London, N1,...","[{'label': 'display', 'lat': 51.53359182524583...",51.533592,-0.108052,NaN,N1,Greater London,Chapel Market Burger Stall (The Naked Sausage),v-1594150008
8,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,508b0ef5e4b0ba32903ec5ed,19 Chapel Market,GB,London,United Kingdom,NaN,288,"[19 Chapel Market, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.533555, 'lng':...",51.533555,-0.109364,NaN,N1 9EZ,Greater London,19 Chapel Market,v-1594150008
9,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4b9baacff964a5205b1836e3,NaN,GB,NaN,United Kingdom,NaN,325,[United Kingdom],"[{'label': 'display', 'lat': 51.53346596325791...",51.533466,-0.110129,NaN,NaN,NaN,Chapel Market DIY,v-1594150008


In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Angel_market.columns if col.startswith('location.')] + ['id']
Angel_market_filtered = Angel_market.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Angel_market_filtered['categories'] = Angel_market_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Angel_market_filtered.columns = [column.split('.')[-1] for column in Angel_market_filtered.columns]

print(Angel_market_filtered.count())
Angel_market_filtered

name                18
categories          18
address             12
cc                  18
city                11
country             18
crossStreet          3
distance            18
formattedAddress    18
labeledLatLngs      18
lat                 18
lng                 18
neighborhood         1
postalCode           9
state               11
id                  18
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Chapel Market,Street Food Gathering,Chapel Market,GB,Islington,United Kingdom,at Liverpool Rd.,236,"[Chapel Market (at Liverpool Rd.), Islington, ...","[{'label': 'display', 'lat': 51.53352287954976...",51.533523,-0.108404,NaN,N1 9QA,Greater London,4ac518eef964a5201ead20e3
1,Islington Farmers’ Market,Farmers Market,Chapel Market,GB,Islington,United Kingdom,Penton St.,326,"[Chapel Market (Penton St.), Islington, Greate...","[{'label': 'display', 'lat': 51.53342836198379...",51.533428,-0.110168,NaN,N1 9PZ,Greater London,4c0b6c36bbc676b0828c4bd5
2,Moro Market Stall,Food Truck,Exmouth Market,GB,Islington,United Kingdom,NaN,75,"[Exmouth Market, Islington, Greater London, Un...","[{'label': 'display', 'lat': 51.53146203403172...",51.531462,-0.106877,NaN,NaN,Greater London,4bffb0f0daf9c9b6c886faef
3,Market News,Smoke Shop,NaN,GB,NaN,United Kingdom,NaN,410,[United Kingdom],"[{'label': 'display', 'lat': 51.53336226940155...",51.533362,-0.111580,NaN,NaN,NaN,4be90afac5220f47606daaca
4,Islington Antiques Market,Flea Market,NaN,GB,NaN,United Kingdom,NaN,416,[United Kingdom],"[{'label': 'display', 'lat': 51.535317, 'lng':...",51.535317,-0.103482,NaN,NaN,NaN,50829f24e4b07dd17bd5da22
5,Bus Stop T @ Baron Street (Chapel Market),Bus Stop,Baron Street,GB,London,United Kingdom,NaN,213,"[Baron Street, London, Greater London, United ...","[{'label': 'display', 'lat': 51.532243, 'lng':...",51.532243,-0.109150,NaN,NaN,Greater London,51d9231be4b00508f5b92f1a
6,Fresh Meat Market,Butcher,35 Chapel Market,GB,London,United Kingdom,NaN,214,"[35 Chapel Market, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.5335461026408,...",51.533546,-0.107833,NaN,N1 9EN,Greater London,4f9c8457e4b0609168f221d9
7,Chapel Market Burger Stall (The Naked Sausage),Burger Joint,Chapel Market,GB,Islington,United Kingdom,NaN,227,"[Chapel Market, Islington, Greater London, N1,...","[{'label': 'display', 'lat': 51.53359182524583...",51.533592,-0.108052,NaN,N1,Greater London,4daae7d64df01c19b193a4cc
8,19 Chapel Market,Residential Building (Apartment / Condo),19 Chapel Market,GB,London,United Kingdom,NaN,288,"[19 Chapel Market, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.533555, 'lng':...",51.533555,-0.109364,NaN,N1 9EZ,Greater London,508b0ef5e4b0ba32903ec5ed
9,Chapel Market DIY,Hardware Store,NaN,GB,NaN,United Kingdom,NaN,325,[United Kingdom],"[{'label': 'display', 'lat': 51.53346596325791...",51.533466,-0.110129,NaN,NaN,NaN,4b9baacff964a5205b1836e3


In [22]:
address = 'Islington, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'market'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5384287 -0.0999051
market .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5384287,-0.0999051&v=20180604&query=market&radius=500&limit=30'

In [24]:
results = requests.get(url).json()

venues = results['response']['venues']

# tranform venues into a dataframe
Isling_market = json_normalize(venues)
print(Isling_market.shape)
Isling_market.head()

(6, 17)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,4bc307bdf8219c74c87bb510,NaN,GB,NaN,United Kingdom,67,[United Kingdom],"[{'label': 'display', 'lat': 51.53868976310612...",51.538690,-0.099023,NaN,NaN,NaN,Market Garden,v-1594150675
1,"[{'id': '4bf58dd8d48988d1f7941735', 'name': 'F...",False,50829f24e4b07dd17bd5da22,NaN,GB,NaN,United Kingdom,425,[United Kingdom],"[{'label': 'display', 'lat': 51.535317, 'lng':...",51.535317,-0.103482,NaN,NaN,NaN,Islington Antiques Market,v-1594150675
2,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",False,4b1bbc9ef964a520fafc23e3,Upper Street,GB,London,United Kingdom,448,"[Upper Street, London, Greater London, N1, Uni...","[{'label': 'display', 'lat': 51.54081408026264...",51.540814,-0.105125,NaN,N1,Greater London,Zaco Mini Market,v-1594150675
3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4f4408f77b0c0a765766d32f,119A Upper Street,GB,London,United Kingdom,169,"[119A Upper Street, London, Greater London, N1...","[{'label': 'display', 'lat': 51.53864817973767...",51.538648,-0.102332,NaN,N1 1QP,Greater London,MarkeTurk,v-1594150675
4,"[{'id': '4bf58dd8d48988d1f9931735', 'name': 'R...",False,4ac518edf964a520b7ac20e3,Upper St.,GB,Islington,United Kingdom,448,"[Upper St., Islington, Greater London, N1 8EF,...","[{'label': 'display', 'lat': 51.53513906955843...",51.535139,-0.103638,Islington,N1 8EF,Greater London,Camden Passage,v-1594150675


In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Isling_market.columns if col.startswith('location.')] + ['id']
Isling_market_filtered = Isling_market.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Isling_market_filtered['categories'] = Isling_market_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Isling_market_filtered.columns = [column.split('.')[-1] for column in Isling_market_filtered.columns]

print(Isling_market_filtered.count())
Isling_market_filtered

name                6
categories          6
address             4
cc                  6
city                4
country             6
distance            6
formattedAddress    6
labeledLatLngs      6
lat                 6
lng                 6
neighborhood        1
postalCode          4
state               4
id                  6
dtype: int64


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Market Garden,Vegetarian / Vegan Restaurant,NaN,GB,NaN,United Kingdom,67,[United Kingdom],"[{'label': 'display', 'lat': 51.53868976310612...",51.538690,-0.099023,NaN,NaN,NaN,4bc307bdf8219c74c87bb510
1,Islington Antiques Market,Flea Market,NaN,GB,NaN,United Kingdom,425,[United Kingdom],"[{'label': 'display', 'lat': 51.535317, 'lng':...",51.535317,-0.103482,NaN,NaN,NaN,50829f24e4b07dd17bd5da22
2,Zaco Mini Market,Food & Drink Shop,Upper Street,GB,London,United Kingdom,448,"[Upper Street, London, Greater London, N1, Uni...","[{'label': 'display', 'lat': 51.54081408026264...",51.540814,-0.105125,NaN,N1,Greater London,4b1bbc9ef964a520fafc23e3
3,MarkeTurk,Grocery Store,119A Upper Street,GB,London,United Kingdom,169,"[119A Upper Street, London, Greater London, N1...","[{'label': 'display', 'lat': 51.53864817973767...",51.538648,-0.102332,NaN,N1 1QP,Greater London,4f4408f77b0c0a765766d32f
4,Camden Passage,Road,Upper St.,GB,Islington,United Kingdom,448,"[Upper St., Islington, Greater London, N1 8EF,...","[{'label': 'display', 'lat': 51.53513906955843...",51.535139,-0.103638,Islington,N1 8EF,Greater London,4ac518edf964a520b7ac20e3
5,BLUEMOON Marketing,Office,313-314 Upper St.,GB,Islington,United Kingdom,222,"[313-314 Upper St., Islington, Greater London,...","[{'label': 'display', 'lat': 51.537561, 'lng':...",51.537561,-0.102806,NaN,N1 2XQ,Greater London,4cb32731eb65b1f7f38b81cd


In [33]:
address = 'Notting Hill, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'market'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5109995 -0.2055267
market .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5109995,-0.2055267&v=20180604&query=market&radius=500&limit=30'

In [34]:
results = requests.get(url).json()

venues = results['response']['venues']

# tranform venues into a dataframe
Notting_market = json_normalize(venues)
print(Notting_market.shape)
Notting_market.head()

(4, 18)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",False,4acaf933f964a52031c320e3,Portobello Rd,GB,London,United Kingdom,NaN,384,"[Portobello Rd, Notting Hill, Greater London, ...","[{'label': 'display', 'lat': 51.51433124729240...",51.514331,-0.204073,W11,Greater London,Portobello Road Market,v-1594151499,48874476
1,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",False,5a09be370531e824a06f46b3,NaN,GB,London,United Kingdom,NaN,383,"[London, Greater London, W11, United Kingdom]","[{'label': 'display', 'lat': 51.514433, 'lng':...",51.514433,-0.205158,W11,Greater London,Portobello And Golborne Market Stop,v-1594151499,NaN
2,"[{'id': '4bf58dd8d48988d116951735', 'name': 'A...",False,5974bfe61f8ed660bbaa9c4e,165-169,GB,London,United Kingdom,Portobello Road,424,"[165-169 (Portobello Road), London, Greater Lo...","[{'label': 'display', 'lat': 51.514763, 'lng':...",51.514763,-0.204507,W11 2DY,Greater London,Red Lion Antiques Market,v-1594151499,NaN
3,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",False,4f576388e4b071452ef31646,NaN,GB,NaN,United Kingdom,NaN,372,[United Kingdom],"[{'label': 'display', 'lat': 51.51407286114021...",51.514073,-0.203383,NaN,NaN,Shepards Markets @ One New Change,v-1594151499,NaN


In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Notting_market.columns if col.startswith('location.')] + ['id']
Notting_market_filtered = Notting_market.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Notting_market_filtered['categories'] = Notting_market_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Notting_market_filtered.columns = [column.split('.')[-1] for column in Notting_market_filtered.columns]

print(Notting_market_filtered.count())
Notting_market_filtered

name                4
categories          4
address             2
cc                  4
city                3
country             4
crossStreet         1
distance            4
formattedAddress    4
labeledLatLngs      4
lat                 4
lng                 4
postalCode          3
state               3
id                  4
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Portobello Road Market,Market,Portobello Rd,GB,London,United Kingdom,NaN,384,"[Portobello Rd, Notting Hill, Greater London, ...","[{'label': 'display', 'lat': 51.51433124729240...",51.514331,-0.204073,W11,Greater London,4acaf933f964a52031c320e3
1,Portobello And Golborne Market Stop,Bus Stop,NaN,GB,London,United Kingdom,NaN,383,"[London, Greater London, W11, United Kingdom]","[{'label': 'display', 'lat': 51.514433, 'lng':...",51.514433,-0.205158,W11,Greater London,5a09be370531e824a06f46b3
2,Red Lion Antiques Market,Antique Shop,165-169,GB,London,United Kingdom,Portobello Road,424,"[165-169 (Portobello Road), London, Greater Lo...","[{'label': 'display', 'lat': 51.514763, 'lng':...",51.514763,-0.204507,W11 2DY,Greater London,5974bfe61f8ed660bbaa9c4e
3,Shepards Markets @ One New Change,Bakery,NaN,GB,NaN,United Kingdom,NaN,372,[United Kingdom],"[{'label': 'display', 'lat': 51.51407286114021...",51.514073,-0.203383,NaN,NaN,4f576388e4b071452ef31646


In [36]:
address = 'Spitalfields, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'market'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5195274 -0.0751701
market .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5195274,-0.0751701&v=20180604&query=market&radius=500&limit=30'

In [37]:
results = requests.get(url).json()

venues = results['response']['venues']

# tranform venues into a dataframe
Spital_market = json_normalize(venues)
print(Spital_market.shape)
Spital_market.head()

(30, 19)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1f7941735', 'name': 'F...",False,4bd2fa6641b9ef3becf7fee5,16 Horner Sq,GB,London,United Kingdom,Brushfield St,21,"[16 Horner Sq (Brushfield St), London, Greater...",NaN,51.519668,-0.075375,NaN,E1 6EW,Greater London,Old Spitalfields Market,v-1594151510,39223713
1,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",False,4ac518ebf964a52047ac20e3,91 Brick Ln,GB,Spitalfields and Banglatown,United Kingdom,NaN,332,"[91 Brick Ln, Spitalfields and Banglatown, Gre...","[{'label': 'display', 'lat': 51.52160002739757...",51.521600,-0.071707,NaN,E1 6QL,Greater London,Brick Lane Market,v-1594151510,NaN
2,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A...",False,4bbdb292f57ba5934051aeb9,Market St.,GB,Spitalfields,United Kingdom,Brushfield St.,124,"[Market St. (Brushfield St.), Spitalfields, Gr...","[{'label': 'display', 'lat': 51.51933710336409...",51.519337,-0.076934,NaN,E1 6AA,Greater London,Spitalfields Arts Market,v-1594151510,NaN
3,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,542d53f2498e38ccd3d1ae30,Goulston St,GB,London,United Kingdom,Wentworth St,269,"[Goulston St (Wentworth St), London, Greater L...","[{'label': 'display', 'lat': 51.51712141163226...",51.517121,-0.075649,Aldgate,NaN,Greater London,Goulston Street Market,v-1594151510,NaN
4,"[{'id': '4bf58dd8d48988d1f7941735', 'name': 'F...",False,4c1cd8a463750f47a025b767,146 Brick Ln,GB,London,United Kingdom,NaN,348,"[146 Brick Ln, London, Greater London, E1 6QL,...","[{'label': 'display', 'lat': 51.52123290606943...",51.521233,-0.070955,NaN,E1 6QL,Greater London,Backyard Market,v-1594151510,NaN


In [38]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Spital_market.columns if col.startswith('location.')] + ['id']
Spital_market_filtered = Spital_market.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Spital_market_filtered['categories'] = Spital_market_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Spital_market_filtered.columns = [column.split('.')[-1] for column in Spital_market_filtered.columns]

print(Spital_market_filtered.count())
Spital_market_filtered

name                30
categories          28
address             23
cc                  30
city                24
country             30
crossStreet          9
distance            30
formattedAddress    30
labeledLatLngs      29
lat                 30
lng                 30
neighborhood         2
postalCode          18
state               24
id                  30
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Old Spitalfields Market,Flea Market,16 Horner Sq,GB,London,United Kingdom,Brushfield St,21,"[16 Horner Sq (Brushfield St), London, Greater...",NaN,51.519668,-0.075375,NaN,E1 6EW,Greater London,4bd2fa6641b9ef3becf7fee5
1,Brick Lane Market,Market,91 Brick Ln,GB,Spitalfields and Banglatown,United Kingdom,NaN,332,"[91 Brick Ln, Spitalfields and Banglatown, Gre...","[{'label': 'display', 'lat': 51.52160002739757...",51.521600,-0.071707,NaN,E1 6QL,Greater London,4ac518ebf964a52047ac20e3
2,Spitalfields Arts Market,Art Gallery,Market St.,GB,Spitalfields,United Kingdom,Brushfield St.,124,"[Market St. (Brushfield St.), Spitalfields, Gr...","[{'label': 'display', 'lat': 51.51933710336409...",51.519337,-0.076934,NaN,E1 6AA,Greater London,4bbdb292f57ba5934051aeb9
3,Goulston Street Market,Food Truck,Goulston St,GB,London,United Kingdom,Wentworth St,269,"[Goulston St (Wentworth St), London, Greater L...","[{'label': 'display', 'lat': 51.51712141163226...",51.517121,-0.075649,Aldgate,NaN,Greater London,542d53f2498e38ccd3d1ae30
4,Backyard Market,Flea Market,146 Brick Ln,GB,London,United Kingdom,NaN,348,"[146 Brick Ln, London, Greater London, E1 6QL,...","[{'label': 'display', 'lat': 51.52123290606943...",51.521233,-0.070955,NaN,E1 6QL,Greater London,4c1cd8a463750f47a025b767
5,Fish Market,Seafood Restaurant,16B New St,GB,London,United Kingdom,NaN,402,"[16B New St, London, Greater London, EC2M 4TR,...","[{'label': 'display', 'lat': 51.51727389118842...",51.517274,-0.079709,NaN,EC2M 4TR,Greater London,5024cfc78055b1005d12d0cb
6,Vintage Market,Market,The Old Truman Brewery,GB,Brick Lane,United Kingdom,Brick Ln.,268,"[The Old Truman Brewery (Brick Ln.), Brick Lan...","[{'label': 'display', 'lat': 51.52085340226896...",51.520853,-0.071929,NaN,E1,Greater London,4d32d7822c76a143573475c7
7,World Food Market,Street Food Gathering,Devonshire Sq,GB,London,United Kingdom,NaN,423,"[Devonshire Sq, London, Greater London, EC2M 4...","[{'label': 'display', 'lat': 51.51623148184633...",51.516231,-0.078217,NaN,EC2M 4TY,Greater London,53e245e1498edef331b8a89c
8,Petticoat Lane Market,Flea Market,Wentworth St,GB,Brick Lane,United Kingdom,NaN,352,"[Wentworth St, Brick Lane, Greater London, E1 ...","[{'label': 'display', 'lat': 51.51635777275846...",51.516358,-0.075101,NaN,E1 7HT,Greater London,4ac518ebf964a52048ac20e3
9,Market Green,Food Truck,64 Middlesex St,GB,NaN,United Kingdom,NaN,15,"[64 Middlesex St, E1 7EZ, United Kingdom]","[{'label': 'display', 'lat': 51.51945220969152...",51.519452,-0.075357,NaN,E1 7EZ,NaN,588b4dac2a19822008c2f21f


## GYMS

In [48]:
#overlapped code for queries "gym", "fitness", and "yoga"
address = 'Angel, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'yoga'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5319458 -0.1061056
yoga .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5319458,-0.1061056&v=20180604&query=yoga&radius=500&limit=30'

In [49]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
Angel_gym = json_normalize(venues)
print(Angel_gym.shape)
Angel_gym.head()

(5, 16)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d131941735', 'name': 'S...",False,50d09e767ab44cf097bb1aee,"Unit 5-8, 28-32 Pentonville Road",GB,London,United Kingdom,47,"[Unit 5-8, 28-32 Pentonville Road, London, Gre...","[{'label': 'display', 'lat': 51.53184101467052...",51.531841,-0.106773,N1 9HJ,Greater London,Soul Power Yoga,v-1594153675
1,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,4e666060b61c804f3f4cd5d5,357 City Rd,GB,Islington,United Kingdom,88,"[357 City Rd, Islington, Greater London, EC1V ...","[{'label': 'display', 'lat': 51.531544, 'lng':...",51.531544,-0.104997,EC1V 1LR,Greater London,Islington Yoga,v-1594153675
2,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,5d7ccf8c20dd070008ac6fce,122 Pentonville Road,GB,London,United Kingdom,467,"[122 Pentonville Road, London, Greater London,...","[{'label': 'display', 'lat': 51.531678, 'lng':...",51.531678,-0.112845,N1 9TT,Greater London,Time + Space Yoga,v-1594153675
3,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,576e668b498e3f8552f45340,Exmouth Market,GB,NaN,United Kingdom,642,"[Exmouth Market, United Kingdom]","[{'label': 'display', 'lat': 51.526337, 'lng':...",51.526337,-0.108269,NaN,NaN,More Yoga Exmouth Market,v-1594153675
4,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,501558ebe4b0748a79d7d26e,122 Pentonville Rd.,GB,Islington,United Kingdom,462,"[122 Pentonville Rd., Islington, Greater Londo...","[{'label': 'display', 'lat': 51.53151568279294...",51.531516,-0.112744,N1 9TT,Greater London,Yogahaven,v-1594153675


In [50]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Angel_gym.columns if col.startswith('location.')] + ['id']
Angel_gym_filtered = Angel_gym.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Angel_gym_filtered['categories'] = Angel_gym_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Angel_gym_filtered.columns = [column.split('.')[-1] for column in Angel_gym_filtered.columns]

print(Angel_gym_filtered.count())
Angel_gym_filtered

name                5
categories          5
address             5
cc                  5
city                4
country             5
distance            5
formattedAddress    5
labeledLatLngs      5
lat                 5
lng                 5
postalCode          4
state               4
id                  5
dtype: int64


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Soul Power Yoga,Spiritual Center,"Unit 5-8, 28-32 Pentonville Road",GB,London,United Kingdom,47,"[Unit 5-8, 28-32 Pentonville Road, London, Gre...","[{'label': 'display', 'lat': 51.53184101467052...",51.531841,-0.106773,N1 9HJ,Greater London,50d09e767ab44cf097bb1aee
1,Islington Yoga,Yoga Studio,357 City Rd,GB,Islington,United Kingdom,88,"[357 City Rd, Islington, Greater London, EC1V ...","[{'label': 'display', 'lat': 51.531544, 'lng':...",51.531544,-0.104997,EC1V 1LR,Greater London,4e666060b61c804f3f4cd5d5
2,Time + Space Yoga,Yoga Studio,122 Pentonville Road,GB,London,United Kingdom,467,"[122 Pentonville Road, London, Greater London,...","[{'label': 'display', 'lat': 51.531678, 'lng':...",51.531678,-0.112845,N1 9TT,Greater London,5d7ccf8c20dd070008ac6fce
3,More Yoga Exmouth Market,Yoga Studio,Exmouth Market,GB,NaN,United Kingdom,642,"[Exmouth Market, United Kingdom]","[{'label': 'display', 'lat': 51.526337, 'lng':...",51.526337,-0.108269,NaN,NaN,576e668b498e3f8552f45340
4,Yogahaven,Yoga Studio,122 Pentonville Rd.,GB,Islington,United Kingdom,462,"[122 Pentonville Rd., Islington, Greater Londo...","[{'label': 'display', 'lat': 51.53151568279294...",51.531516,-0.112744,N1 9TT,Greater London,501558ebe4b0748a79d7d26e


In [57]:
#overlapped code for queries "gym", "fitness", and "yoga"
address = 'Islington, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'yoga'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5384287 -0.0999051
yoga .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5384287,-0.0999051&v=20180604&query=yoga&radius=500&limit=30'

In [58]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
Isling_gym = json_normalize(venues)
print(Isling_gym.shape)
Isling_gym.head()

(3, 17)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,5d1ce5316d54f80023b28d6b,21 Essex Road,GB,London,United Kingdom,NaN,215,"[21 Essex Road, London, Greater London, N1 8LN...","[{'label': 'display', 'lat': 51.5367, 'lng': -...",51.536700,-0.101300,N1 8LN,Greater London,More Yoga Angel,v-1594154133
1,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,4ec7bbe0775bf65c03b638b3,199 Upper Street,GB,London,United Kingdom,NaN,508,"[199 Upper Street, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.54253637163741...",51.542536,-0.103122,N1 1RQ,Greater London,Pop Up Yoga,v-1594154133
2,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,4c45a899429a0f47ec624a1e,255-257 Liverpool Road,GB,London,United Kingdom,Islington,484,"[255-257 Liverpool Road (Islington), London, G...","[{'label': 'display', 'lat': 51.54018901991176...",51.540189,-0.106313,N1 1LX,Greater London,Yogabase,v-1594154133


In [59]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Isling_gym.columns if col.startswith('location.')] + ['id']
Isling_gym_filtered = Isling_gym.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Isling_gym_filtered['categories'] = Isling_gym_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Isling_gym_filtered.columns = [column.split('.')[-1] for column in Isling_gym_filtered.columns]

print(Isling_gym_filtered.count())
Isling_gym_filtered

name                3
categories          3
address             3
cc                  3
city                3
country             3
crossStreet         1
distance            3
formattedAddress    3
labeledLatLngs      3
lat                 3
lng                 3
postalCode          3
state               3
id                  3
dtype: int64


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,More Yoga Angel,Yoga Studio,21 Essex Road,GB,London,United Kingdom,NaN,215,"[21 Essex Road, London, Greater London, N1 8LN...","[{'label': 'display', 'lat': 51.5367, 'lng': -...",51.536700,-0.101300,N1 8LN,Greater London,5d1ce5316d54f80023b28d6b
1,Pop Up Yoga,Yoga Studio,199 Upper Street,GB,London,United Kingdom,NaN,508,"[199 Upper Street, London, Greater London, N1 ...","[{'label': 'display', 'lat': 51.54253637163741...",51.542536,-0.103122,N1 1RQ,Greater London,4ec7bbe0775bf65c03b638b3
2,Yogabase,Yoga Studio,255-257 Liverpool Road,GB,London,United Kingdom,Islington,484,"[255-257 Liverpool Road (Islington), London, G...","[{'label': 'display', 'lat': 51.54018901991176...",51.540189,-0.106313,N1 1LX,Greater London,4c45a899429a0f47ec624a1e


In [66]:
#overlapped code for queries "gym", "fitness", and "yoga"
address = 'Notting Hill, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'yoga'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5109995 -0.2055267
yoga .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5109995,-0.2055267&v=20180604&query=yoga&radius=500&limit=30'

In [67]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
Notting_gym = json_normalize(venues)
print(Notting_gym.shape)
Notting_gym.head()

(1, 17)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,[],False,562a0d68498e4c9fb03a6cb6,56 Pembridge Villas,GB,London,United Kingdom,523,"[56 Pembridge Villas, London, Greater London, ...","[{'label': 'display', 'lat': 51.5118252, 'lng'...",51.511825,-0.198082,W11 3EG,Greater London,"YOGA I AM - Retreats, Ibiza,Goa, UK",v-1594154448,143782459


In [68]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Notting_gym.columns if col.startswith('location.')] + ['id']
Notting_gym_filtered = Notting_gym.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Notting_gym_filtered['categories'] = Notting_gym_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Notting_gym_filtered.columns = [column.split('.')[-1] for column in Notting_gym_filtered.columns]

print(Notting_gym_filtered.count())
Notting_gym_filtered

name                1
categories          0
address             1
cc                  1
city                1
country             1
distance            1
formattedAddress    1
labeledLatLngs      1
lat                 1
lng                 1
postalCode          1
state               1
id                  1
dtype: int64


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,"YOGA I AM - Retreats, Ibiza,Goa, UK",None,56 Pembridge Villas,GB,London,United Kingdom,523,"[56 Pembridge Villas, London, Greater London, ...","[{'label': 'display', 'lat': 51.5118252, 'lng'...",51.511825,-0.198082,W11 3EG,Greater London,562a0d68498e4c9fb03a6cb6


In [89]:
#overlapped code for queries "gym", "fitness", and "yoga"
address = 'Shoreditch, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'gym'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5266694 -0.0798926
gym .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5266694,-0.0798926&v=20180604&query=gym&radius=500&limit=30'

In [90]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
Shore_gym = json_normalize(venues)
print(Shore_gym.shape)
Shore_gym.head()

(16, 17)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,52f339e8498ea23d1c2d31d8,100 Shoreditch High St.,GB,London,United Kingdom,199,"[100 Shoreditch High St., London, Greater Lond...","[{'label': 'display', 'lat': 51.52562613460337...",51.525626,-0.077550,NaN,Greater London,Ace Hotel Shoreditch Gym,v-1594163994,NaN
1,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,560fe3d9498e9d99c49fd501,"1-6, Bateman's Row",GB,London,United Kingdom,147,"[1-6, Bateman's Row, London, Greater London, E...","[{'label': 'display', 'lat': 51.525383, 'lng':...",51.525383,-0.079369,EC2A 3HH,Greater London,Nuffield Health Fitness & Wellbeing Gym,v-1594163994,NaN
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4eb518ccf790d87d02905b82,91 City Road,GB,Islington,United Kingdom,535,"[91 City Road, Islington, Greater London, EC1Y...","[{'label': 'display', 'lat': 51.5253612775441,...",51.525361,-0.087333,EC1Y 1AF,Greater London,Bezier Apartments Gym,v-1594163994,NaN
3,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,5a0ee30eca18ea3297314d9a,Willow Street,GB,London,United Kingdom,276,"[Willow Street, London, Greater London, EC2A 4...","[{'label': 'display', 'lat': 51.525287, 'lng':...",51.525287,-0.083216,EC2A 4BH,Greater London,Nobu Hotel Gym,v-1594163994,NaN
4,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,5072f46ce4b0e156bba56520,NaN,GB,NaN,United Kingdom,353,[United Kingdom],"[{'label': 'display', 'lat': 51.52969260065398...",51.529693,-0.078334,NaN,NaN,IQ Hoxton Gym,v-1594163994,NaN


In [91]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Shore_gym.columns if col.startswith('location.')] + ['id']
Shore_gym_filtered = Shore_gym.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Shore_gym_filtered['categories'] = Shore_gym_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Shore_gym_filtered.columns = [column.split('.')[-1] for column in Shore_gym_filtered.columns]

print(Shore_gym_filtered.count())
Shore_gym_filtered

name                16
categories          16
address             10
cc                  16
city                10
country             16
distance            16
formattedAddress    16
labeledLatLngs      16
lat                 16
lng                 16
postalCode           9
state               10
id                  16
dtype: int64


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Ace Hotel Shoreditch Gym,Gym / Fitness Center,100 Shoreditch High St.,GB,London,United Kingdom,199,"[100 Shoreditch High St., London, Greater Lond...","[{'label': 'display', 'lat': 51.52562613460337...",51.525626,-0.077550,NaN,Greater London,52f339e8498ea23d1c2d31d8
1,Nuffield Health Fitness & Wellbeing Gym,Gym / Fitness Center,"1-6, Bateman's Row",GB,London,United Kingdom,147,"[1-6, Bateman's Row, London, Greater London, E...","[{'label': 'display', 'lat': 51.525383, 'lng':...",51.525383,-0.079369,EC2A 3HH,Greater London,560fe3d9498e9d99c49fd501
2,Bezier Apartments Gym,Gym,91 City Road,GB,Islington,United Kingdom,535,"[91 City Road, Islington, Greater London, EC1Y...","[{'label': 'display', 'lat': 51.5253612775441,...",51.525361,-0.087333,EC1Y 1AF,Greater London,4eb518ccf790d87d02905b82
3,Nobu Hotel Gym,Gym / Fitness Center,Willow Street,GB,London,United Kingdom,276,"[Willow Street, London, Greater London, EC2A 4...","[{'label': 'display', 'lat': 51.525287, 'lng':...",51.525287,-0.083216,EC2A 4BH,Greater London,5a0ee30eca18ea3297314d9a
4,IQ Hoxton Gym,Gym,NaN,GB,NaN,United Kingdom,353,[United Kingdom],"[{'label': 'display', 'lat': 51.52969260065398...",51.529693,-0.078334,NaN,NaN,5072f46ce4b0e156bba56520
5,Scape Shoreditch Gym,Residential Building (Apartment / Condo),NaN,GB,NaN,United Kingdom,477,[United Kingdom],"[{'label': 'display', 'lat': 51.52763818482078...",51.527638,-0.086612,NaN,NaN,55f45b1c498ef095942e779f
6,Paragon Gym,Gym / Fitness Center,9 Boundary Street,GB,London,United Kingdom,241,"[9 Boundary Street, London, Greater London, E1...","[{'label': 'display', 'lat': 51.5259208516516,...",51.525921,-0.076626,E1 7JE,Greater London,4ac518faf964a52038b020e3
7,OBSESSIVE GYM DISORDER,Gym,22 Corsham Street,GB,London,United Kingdom,477,"[22 Corsham Street, London, Greater London, N1...","[{'label': 'display', 'lat': 51.5278095, 'lng'...",51.527809,-0.086541,N1 6DR,Greater London,582db343f776b0101dd5ec22
8,Inmarsat Gym,Gym,NaN,GB,NaN,United Kingdom,489,[United Kingdom],"[{'label': 'display', 'lat': 51.52546638255596...",51.525466,-0.086691,NaN,NaN,5100203ae4b000bd979c34bf
9,IQ Shoreditch Gym,Gym,NaN,GB,NaN,United Kingdom,568,[United Kingdom],"[{'label': 'display', 'lat': 51.528124, 'lng':...",51.528124,-0.087759,NaN,NaN,52a63fb311d21f220b240632


In [86]:
#overlapped code for queries "gym", "fitness", and "yoga"
address = 'Spitalfields, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'yoga'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

51.5195274 -0.0751701
yoga .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=CH2SVOH5T2VO1MDUAEZZTXG3J4TDIKBQTMLQLR215DMQJXM3&client_secret=OGUAYNKMFPZZSE3ZGDOB21S10RVFXZ1VR1FC1DPAOGJUFXP0&ll=51.5195274,-0.0751701&v=20180604&query=yoga&radius=500&limit=30'

In [87]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
Spital_gym = json_normalize(venues)
print(Spital_gym.shape)
Spital_gym.head()

(3, 16)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,5063fc6ee4b080e22cbe00e8,30 Hanbury St,GB,London,United Kingdom,209,"[30 Hanbury St, London, Greater London, E1 6QR...","[{'label': 'display', 'lat': 51.5204550374872,...",51.520455,-0.072543,E1 6QR,Greater London,Bodywise Yoga & Natural Health Centre,v-1594155461
1,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,5975cf7d4a7aae59a85f470b,7 Harrow Place,GB,London,United Kingdom,275,"[7 Harrow Place, London, Greater London, E1 7D...","[{'label': 'display', 'lat': 51.51743583353309...",51.517436,-0.077301,E1 7DB,Greater London,Prana Yoga Space,v-1594155461
2,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",False,4d4c0bb6fe85a1cd4ddf98bf,Yoga Studio,GB,London,United Kingdom,524,"[Yoga Studio, London, Greater London, United K...","[{'label': 'display', 'lat': 51.52422608756188...",51.524226,-0.075738,NaN,Greater London,Yoga Place,v-1594155461


In [88]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in Spital_gym.columns if col.startswith('location.')] + ['id']
Spital_gym_filtered = Spital_gym.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
Spital_gym_filtered['categories'] = Spital_gym_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
Spital_gym_filtered.columns = [column.split('.')[-1] for column in Spital_gym_filtered.columns]

print(Spital_gym_filtered.count())
Spital_gym_filtered

name                3
categories          3
address             3
cc                  3
city                3
country             3
distance            3
formattedAddress    3
labeledLatLngs      3
lat                 3
lng                 3
postalCode          2
state               3
id                  3
dtype: int64


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Bodywise Yoga & Natural Health Centre,Yoga Studio,30 Hanbury St,GB,London,United Kingdom,209,"[30 Hanbury St, London, Greater London, E1 6QR...","[{'label': 'display', 'lat': 51.5204550374872,...",51.520455,-0.072543,E1 6QR,Greater London,5063fc6ee4b080e22cbe00e8
1,Prana Yoga Space,Yoga Studio,7 Harrow Place,GB,London,United Kingdom,275,"[7 Harrow Place, London, Greater London, E1 7D...","[{'label': 'display', 'lat': 51.51743583353309...",51.517436,-0.077301,E1 7DB,Greater London,5975cf7d4a7aae59a85f470b
2,Yoga Place,Yoga Studio,Yoga Studio,GB,London,United Kingdom,524,"[Yoga Studio, London, Greater London, United K...","[{'label': 'display', 'lat': 51.52422608756188...",51.524226,-0.075738,NaN,Greater London,4d4c0bb6fe85a1cd4ddf98bf


## keeping real

In [ ]:
=2+2